In [1]:
import warnings
warnings.filterwarnings('ignore')
import os
import sys
sys.path.insert(0, '../scripts')
from l5kit_modified.l5kit_modified import FramesDataset, create_chopped_dataset, AgentDatasetModified, get_agent_indices_set
from kalman import KalmanTrackerPredictor
from lane_processing import *
import pickle
from l5kit.evaluation import write_pred_csv, compute_metrics_csv
from l5kit.evaluation.metrics import neg_multi_log_likelihood
from l5kit.data import LocalDataManager, ChunkedDataset
from pathlib import Path
import numpy as np
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import seaborn as sns
from copy import deepcopy

from pytz import timezone
from datetime import datetime, timedelta
from l5kit.data.filter import filter_tl_faces_by_status
from workalendar.usa import CaliforniaSanFrancisco
import pandas as pd
import json
from django.core.serializers.json import DjangoJSONEncoder

import imutils

from collections import defaultdict, deque
import gc
import multiprocessing as mp
import tracemalloc
import linecache
from functools import partial
tracemalloc.start()

def display_top(snapshot, key_type='lineno', limit=20):
    snapshot = snapshot.filter_traces((
        tracemalloc.Filter(False, "<frozen importlib._bootstrap>"),
        tracemalloc.Filter(False, "<unknown>"),
    ))
    top_stats = snapshot.statistics(key_type)

    print("Top %s lines" % limit)
    for index, stat in enumerate(top_stats[:limit], 1):
        frame = stat.traceback[0]
        # replace "/path/to/module/file.py" with "module/file.py"
        filename = os.sep.join(frame.filename.split(os.sep)[-2:])
        print("#%s: %s:%s: %.1f KiB"
              % (index, filename, frame.lineno, stat.size / 1024))
        line = linecache.getline(frame.filename, frame.lineno).strip()
        if line:
            print('    %s' % line)

    other = top_stats[limit:]
    if other:
        size = sum(stat.size for stat in other)
        print("%s other: %.1f KiB" % (len(other), size / 1024))
    total = sum(stat.size for stat in top_stats)
    print("Total allocated size: %.1f KiB" % (total / 1024))



import plotly.express as px


snapshot = tracemalloc.take_snapshot()
# display_top(snapshot)

%matplotlib inline
    
os.environ["L5KIT_DATA_FOLDER"] = "../input/"
CAR_CLASS = 3
BIKE_CLASS = 10
ALL_WHEELS_CLASS = -1
TL_GREEN_COLOR = 1
TL_RED_COLOR = 0
TL_YELLOW_COLOR = 0
VIS_WIP = False
LANE_IDLE_CODE = 0
LANE_ON_MOVE_CODE = 1

---------------------------
Comparing filtered vs. orig datasets

In [2]:
dm = LocalDataManager(None)

In [3]:
train_zarr = ChunkedDataset(dm.require('scenes/train.zarr')).open()
train_zarr_filtered = ChunkedDataset(dm.require('scenes/train_full_filtered_min_frame_history_4_min_frame_future_1.zarr')).open()

In [4]:
len(train_zarr_filtered.agents)/len(train_zarr.agents)

0.7927587507295284

------------------------------------

Checking junction info from the semantic map

In [5]:
semantic_map_path = dm.require(semantic_map_key)
proto_API = MapAPI(semantic_map_path, world_to_ecef)

In [6]:
# lanes_crosswalks = precompute_map_elements(proto_API)
# with open('../input/lanes_crosswalks.pkl', 'wb') as f:
#     pickle.dump(lanes_crosswalks, f)
with open('../input/lanes_crosswalks.pkl', 'rb') as f:
    lanes_crosswalks = pickle.load(f)

In [7]:
def get_lanes_dict_and_id_mapping(filter_function):
    lanes = dict()
    lanes_indices = [idx for idx, lane_id in enumerate(lanes_crosswalks['lanes']['ids'])
                     if filter_function(lane_id)]
        
    for key, values in lanes_crosswalks['lanes'].items():
        if len(lanes_crosswalks['lanes']['ids']) == len(values):
            lanes[key] = values[lanes_indices] if isinstance(values, (np.ndarray, np.generic)) else [values[i] for i in lanes_indices]

    lane_id_2_idx = {lane_id: i for i, lane_id in enumerate(lanes['ids'])}
    return lanes, lane_id_2_idx

In [8]:
lanes_not_bike, lane_id_2_idx_not_bike = get_lanes_dict_and_id_mapping(filter_function=lambda x: not proto_API.is_bike_only_lane(x))
lanes_bike, lane_id_2_idx_bike = get_lanes_dict_and_id_mapping(filter_function=lambda x: proto_API.is_bike_only_lane(x))
lane_id_2_idx = {lane_id: i for i, lane_id in enumerate(lanes_crosswalks['lanes']['ids'])}

lane_adj_list_forward_bike, lane_adj_list_backward_bike, lane_adj_list_right_bike, lane_adj_list_left_bike = precompute_lane_adjacencies(lane_id_2_idx_bike, proto_API)
lane_adj_list_forward_not_bike, lane_adj_list_backward_not_bike, lane_adj_list_right_not_bike, lane_adj_list_left_not_bike = precompute_lane_adjacencies(lane_id_2_idx_not_bike, proto_API)

In [9]:
# right turn under red fixex (initially assumed additional green arrow face)
# example: parent m+dt, 
#     leave tl control connection to li+4 (if more, the first one must be the closest to the turn lane); 
#     remove connection to Rvxh
#     lanes to yield on red-light turn:  ['nwfo', 'AAR1']]
turn_under_red_fixes = [['m+dt', ['li+4'], 'Rvxh', ['nwfo', 'AAR1']],
                        ['/E65', ['u016'], 'Waxo', ['qSSB', 'pQYu']],
                        ['T1zT', ['nicu'], 'hE9x', ['li+4']],
                        ['RRp7', ['9qCn'], 'le35', ['SYA4']],
                        ['/zLv', ['Pl2o'], '3KTc', ['4LEI']],
                        ['VUqQ', ['YeLI'], 'fMAY', ['/9cr', 'KtDh']],
                        ['cKUZ', ['x+C2'], 'b2Km', ['mTBO', 'JDFg']],
                        ['Myw0', ['ZjWH', '/9cr', 'biWH', '6iWH'], 'KHxd', ['x+C2', 'SE96']],
                        ['89HH', ['RbC9'], '5z+h', ['wqVO', 'yU+6']],
                       ['jMm1', ['KTUI'], 'wTjJ', ['SOK7', 'JhJI', 'UCr8']],
                        ['YkSE', ['SOK7'], '+Klg', ['RbC9', 'JAlK']],
                        ['ake8', ['z5yd'], '+SHD', ['xb0W', '/aG3', 'A+L+']],
                        ['vijX', ['A+L+'], 'H09x', ['u016', 'wc+d']],
                        ['hINg', ['2OcN', 'XOcN'], 'VPcN', ['z5yd', '8weJ']],
                        ['aVLS', ['zojk'], '+B39', ['I4aR']],
                        ['CVyh', ['1WGF', 'WWGF', 'wtG4', 'PuG4'], 'UXGF', ['zojk']],
                        ['emZY', ['84vV'], 'FWhR', []],
                        ['YCLp', ['zvWg'], 'L5L1', ['84vV']],
                        ['TnVm', ['nwfo'], 'hQc1', ['78bQ', 'M3Fu']],
                        ['DvGy', ['qgZI'], 'UB+4', ['+P/J', '9qCn']],
                        ['b75w', ['SYA4'], '4gKt', ['Q20G']]
                       ]

In [10]:
fix_i_2_junction_bro = dict()
red_turn_lane_2_yield_lanes = dict()
added_lanes = set()

def get_helping_angle(vector_1):
    vector_0 = np.array([1, 0])
    angle_cos = np.dot(vector_0, vector_1)/np.linalg.norm(vector_1)
    alpha = np.arccos(angle_cos)    
    if vector_1[1] < 0:
        alpha = 2*np.pi - alpha
    return alpha

def get_connecting_coordinates(start_point_coord, end_point_coord, total_dist=None, step_m=2):
    vector_1 = end_point_coord - start_point_coord
    helping_angle = get_helping_angle(vector_1)
    sin_, cos_ = np.sin(helping_angle), np.cos(helping_angle)
    if total_dist is None:
        total_dist = np.hypot(start_point_coord[0] - end_point_coord[0], start_point_coord[1] - end_point_coord[1])
    dist = 0
    current_point_coords = start_point_coord
    all_coords = [start_point_coord.copy()]
    while dist <= total_dist:
        current_point_coords += step_m*np.array([cos_, sin_])
        dist += step_m
        all_coords.append(current_point_coords.copy())
    return np.vstack(all_coords)
    
lane_id_2_idx_specialized = [lane_id_2_idx_bike, lane_id_2_idx_not_bike]
lane_adj_list_forward_specialized = [lane_adj_list_forward_bike, lane_adj_list_forward_not_bike]
lane_adj_list_backward_specialized = [lane_adj_list_backward_bike, lane_adj_list_backward_not_bike]
lane_adj_list_left_specialized = [lane_adj_list_left_bike, lane_adj_list_left_not_bike]
lane_adj_list_right_specialized = [lane_adj_list_right_bike, lane_adj_list_right_not_bike]
lanes_specialized = [lanes_bike, lanes_not_bike]

for fix_i, turn_under_red_fix in enumerate(turn_under_red_fixes):
    parent, lanes_active, lane_to_remove, turn_yield_lanes = turn_under_red_fix
    lane_spec = int(parent in lane_id_2_idx_not_bike)
    lane_id_2_idx_specialized[lane_spec][f'lane{2*fix_i}'] = len(lane_adj_list_forward_specialized[lane_spec])
    lane_adj_list_forward_specialized[lane_spec].append([lane_to_remove])
    lane_adj_list_backward_specialized[lane_spec].append([parent])
    lane_adj_list_left_specialized[lane_spec].append([])
    lane_adj_list_right_specialized[lane_spec].append([])

    lane_id_2_idx_specialized[lane_spec][f'lane{2*fix_i + 1}'] = len(lane_adj_list_forward_specialized[lane_spec])
    lane_adj_list_forward_specialized[lane_spec].append(lanes_active)
    lane_adj_list_backward_specialized[lane_spec].append([parent])
    lane_adj_list_left_specialized[lane_spec].append([])
    lane_adj_list_right_specialized[lane_spec].append([])

    lane_adj_list_forward_specialized[lane_spec][lane_id_2_idx_specialized[lane_spec][parent]] = [f'lane{2*fix_i}', f'lane{2*fix_i + 1}']

    for lane_id_left in lanes_active:
        lane_adj_list_backward_specialized[lane_spec][lane_id_2_idx_specialized[lane_spec][lane_id_left]] = [f'lane{2*fix_i + 1}'] + [x for x in lane_adj_list_backward_specialized[lane_spec][lane_id_2_idx_specialized[lane_spec][lane_id_left]] if x != parent]
    lane_adj_list_backward_specialized[lane_spec][lane_id_2_idx_specialized[lane_spec][lane_to_remove]] = [f'lane{2*fix_i}'] + [x for x in lane_adj_list_backward_specialized[lane_spec][lane_id_2_idx_specialized[lane_spec][lane_to_remove]] if x != parent]

    assert len(lanes_specialized[lane_spec]['center_line'][lane_id_2_idx_specialized[lane_spec][lane_to_remove]]) >= 4
    assert len(lanes_specialized[lane_spec]['center_line'][lane_id_2_idx_specialized[lane_spec][lanes_active[0]]]) >= 5
    
    lanes_specialized[lane_spec]['ids'].extend([f'lane{2*fix_i}', f'lane{2*fix_i + 1}'])
    lanes_crosswalks['lanes']['ids'].extend([f'lane{2*fix_i}', f'lane{2*fix_i + 1}'])
    added_lanes.update({f'lane{2*fix_i}', f'lane{2*fix_i + 1}'})
    
    # last 3 points of parent to be forked
    parent_center_line = lanes_crosswalks['lanes']['center_line'][lane_id_2_idx[parent]]
    init_parent_last_coord = parent_center_line[-1].copy()
    lanes_specialized[lane_spec]['center_line'][lane_id_2_idx_specialized[lane_spec][parent]] = parent_center_line[:max(1, len(parent_center_line)-3)]
    lanes_crosswalks['lanes']['center_line'][lane_id_2_idx[parent]] = parent_center_line[:max(1, len(parent_center_line)-3)]
    new_parent_last_coord = lanes_specialized[lane_spec]['center_line'][lane_id_2_idx_specialized[lane_spec][parent]][-1].copy()
    total_parent_segment_len = np.hypot(new_parent_last_coord[0] - init_parent_last_coord[0], 
                                        new_parent_last_coord[1] - init_parent_last_coord[1])
    # the turn lane to remove
    directional_point_for_fork = lanes_specialized[lane_spec]['center_line'][lane_id_2_idx_specialized[lane_spec][lane_to_remove]][3]
    turn_fork_center_line = get_connecting_coordinates(new_parent_last_coord, directional_point_for_fork, 
                                                       total_dist=total_parent_segment_len)
                               
    lanes_specialized[lane_spec]['center_line'].append(turn_fork_center_line)
    lanes_crosswalks['lanes']['center_line'].append(turn_fork_center_line) 
    # dummy boarder in order not to break vis
    lanes_specialized[lane_spec]['xy_left_'].append(turn_fork_center_line)
    lanes_specialized[lane_spec]['xy_right_'].append(turn_fork_center_line)
    # shouldn't be used from lanes_crosswalks['lanes']
    
    # the first lane to leave tl control must be the closest one to the turn-lane
    directional_point_for_fork = lanes_specialized[lane_spec]['center_line'][lane_id_2_idx_specialized[lane_spec][lanes_active[0]]][4]
    new_parent_last_coord = lanes_specialized[lane_spec]['center_line'][lane_id_2_idx_specialized[lane_spec][parent]][-1].copy()
    straight_center_line = get_connecting_coordinates(new_parent_last_coord, directional_point_for_fork, 
                                                       total_dist=total_parent_segment_len)
                               
    lanes_specialized[lane_spec]['center_line'].append(straight_center_line)  
    lanes_crosswalks['lanes']['center_line'].append(straight_center_line)  
    # dummy boarder in order not to break vis
    lanes_specialized[lane_spec]['xy_left_'].append(straight_center_line)
    lanes_specialized[lane_spec]['xy_right_'].append(straight_center_line)

    fix_i_2_junction_bro[fix_i] = parent
    
    red_turn_lane_2_yield_lanes[f'lane{2*fix_i}'] = turn_yield_lanes

In [11]:
def get_kd_tree_and_idx_map(lanes):
    kd_tree = KDTree(np.concatenate(lanes['center_line'], axis=0))
    kd_idx_2_lane_id_idx = []
    for lane_id, center_line in zip(lanes['ids'],
                                    lanes['center_line']):
        next_entries = [(lane_id, i) for i in range(len(center_line))]
        kd_idx_2_lane_id_idx.extend(next_entries)
    return kd_tree, kd_idx_2_lane_id_idx    
    
kd_tree_bike, kd_idx_2_lane_id_idx_bike = get_kd_tree_and_idx_map(lanes_bike)
kd_tree_not_bike, kd_idx_2_lane_id_idx_not_bike = get_kd_tree_and_idx_map(lanes_not_bike)
kd_tree, kd_idx_2_lane_id_idx = get_kd_tree_and_idx_map(lanes_crosswalks['lanes'])

In [12]:
tl_control_2_junctions = defaultdict(list)
junction_2_tl_controls = defaultdict(list)
lane_2_junctions = defaultdict(list)
junction_2_lanes = defaultdict(list)
for element in tqdm(proto_API):
    if MapAPI.is_junction(element):
        junction_element_id = MapAPI.id_as_str(element.id)
        traffic_control_elements = proto_API.get_traffic_control_elements_at_junction(junction_element_id)
        junction_2_tl_controls[junction_element_id].extend(traffic_control_elements)
        for tl_control in traffic_control_elements:
            tl_control_2_junctions[tl_control].append(junction_element_id)
            
        lanes_related_to_junction = proto_API.get_all_lanes_at_junction(junction_element_id)
        for lane in lanes_related_to_junction:
            junction_2_lanes[junction_element_id].append(lane)
            lane_2_junctions[lane].append(junction_element_id)
# to address potential duplications
for lane_id, junctions in lane_2_junctions.items():
    lane_2_junctions[lane_id] = list(set(junctions))
for junction_id, lanes in junction_2_lanes.items():
    junction_2_lanes[junction_id] = list(set(lanes))
    
for fix_i in range(len(turn_under_red_fixes)):
    junction_neighb = fix_i_2_junction_bro[fix_i]
    lane_2_junctions[f'lane{2*fix_i}'] = lane_2_junctions[junction_neighb].copy()
    lane_2_junctions[f'lane{2*fix_i + 1}'] = lane_2_junctions[junction_neighb].copy()
    for junction_id in lane_2_junctions[junction_neighb]:
        junction_2_lanes[junction_id].extend([f'lane{2*fix_i}', f'lane{2*fix_i + 1}'])

In [13]:
# traffic_light_ids_all = set(eval_zarr.tl_faces['traffic_light_id'])
# with open('traffic_light_ids_all.pkl', 'wb') as f:
#     pickle.dump(traffic_light_ids_all, f)
with open('traffic_light_ids_all.pkl', 'rb') as f:
    traffic_light_ids_all = pickle.load(f)

In [14]:
# traffic_faces_ids_all = set(eval_zarr.tl_faces['face_id'])
# with open('traffic_faces_ids_all.pkl', 'wb') as f:
#     pickle.dump(traffic_faces_ids_all, f)
with open('traffic_faces_ids_all.pkl', 'rb') as f:
    traffic_faces_ids_all = pickle.load(f)

In [15]:
traffic_light_id_2_coord = dict()
tl_ids_without_sem_map_info = []
for el_id in traffic_light_ids_all:
    if el_id in proto_API.ids_to_el:
        coordinates = proto_API.get_traffic_light_coords(el_id)['xyz']
        traffic_light_id_2_coord[el_id] = coordinates[:, :2].mean(axis=0)
    else:
        tl_ids_without_sem_map_info.append(el_id)


added_lanes_tl_coord_tl_id = [[['Myw0'], (684, -407)],
                       [['YCLp', 'H1RI'], (630, -2242)],
                       [['m1RI'], (630, -2242)],
                       [['ii1t'], (634, -2268)],
                       [['TnVm', 'urF8', 'PrF8'], (635, -1355)],
                       [['wqF8'], (636, -1358)],
                       [['T1zT', 'udk+', 'Pdk+'], (672, -1342)],
                       [['wck+'], (667, -1339)],
                       [['wmAx', 'RRp7'], (465, -160)],
                       [['CPtm'], (439, -190)],
                       [['8OiE', 'FHu2'], (249, 82)],
                       [['6PiE'], (213, 56)],
                       [['WlSE', 'pXNd', 'YkSE'],  (202, 63)],
                       [['RLKQ'], (-35, 350)],
                       [['jUyh', 'CVyh'], (-758, 1127)],
                       [['57g+'], (-733, 1125)],
                       [['W9g+', '38g+', 'Y8g+'], (-735, 1122)],
                       [['mzvz'], (-491, 896)],
                       [['F0vz', 'k0vz'], (-488, 894)],
                       [['90Lv'], (-527, 874)],
                       [['e0Lv', '/zLv'], (-530, 876)],
                              [['T3uS', 'y3uS'], (-973.5, 1360)],
                             [['RJu6'],  (-974, 1359)],
                              [['8V6+'], (-950, 1440)], 
                              [['LXvy'], (-510, 875)],
                             [['YVHi'], (-516, 901)],
                              [['9ie8'], (741, -463)]
                             ]  
for i in range(len(added_lanes_tl_coord_tl_id)):
    added_tl_id = f'Luda_{i}'
    added_lanes_tl_coord_tl_id[i].append(added_tl_id)
    traffic_light_id_2_coord[added_tl_id] = np.array(added_lanes_tl_coord_tl_id[i][1])
    traffic_light_ids_all.add(added_tl_id)
    for lane_id in added_lanes_tl_coord_tl_id[i][0]:
        tl_control_2_junctions[added_tl_id].extend(lane_2_junctions[lane_id])
        for junction in lane_2_junctions[lane_id]:
            junction_2_tl_controls[junction].append(added_tl_id)

In [16]:
tl_ids_without_sem_map_info

['Ae1d']

In [17]:
tl_control_2_close_lanes = defaultdict(list)
for tl_control, junctions in tl_control_2_junctions.items():
    if tl_control in traffic_light_ids_all or tl_control in traffic_faces_ids_all:
        for junction in junctions:
            tl_control_2_close_lanes[tl_control].extend(junction_2_lanes[junction])
            # predecessor lanes
            for junction_lane in junction_2_lanes[junction]:
                if junction_lane in lane_id_2_idx_bike:
                    tl_control_2_close_lanes[tl_control].extend(lane_adj_list_backward_bike[lane_id_2_idx_bike[junction_lane]])
                elif junction_lane in lane_id_2_idx_not_bike:
                    tl_control_2_close_lanes[tl_control].extend(lane_adj_list_backward_not_bike[lane_id_2_idx_not_bike[junction_lane]])
        tl_control_2_close_lanes[tl_control] = list(set(tl_control_2_close_lanes[tl_control]))
    
lane_2_close_tl_controls = defaultdict(list)
for tl_control, close_lanes in tl_control_2_close_lanes.items():
    for lane in close_lanes:
        lane_2_close_tl_controls[lane].append(tl_control)
        
lane_2_tl_related_lanes_wip = defaultdict(set)
for lane_id, close_tl_controls in lane_2_close_tl_controls.items():
    for traffic_control_id in close_tl_controls:
        lane_2_tl_related_lanes_wip[lane_id].update(tl_control_2_close_lanes[traffic_control_id])

In [18]:
len(lane_2_tl_related_lanes_wip), len(lane_id_2_idx_bike), len(lane_id_2_idx_not_bike)

(881, 540, 8007)

In [19]:
lane_ids_with_wrong_tl_associations = {'i2bp', 'AxmM', 'LXvy', 'YVHi', '8V6+', 'Bm9O', '9ie8'}

In [20]:
tl_added_lane_id_2_tl_id = dict()
for lanes, _, tl_id in added_lanes_tl_coord_tl_id:
    for lane_id in lanes:
        assert lane_id not in tl_added_lane_id_2_tl_id
        tl_added_lane_id_2_tl_id[lane_id] = tl_id
        lane_2_close_tl_controls[lane_id].append(tl_id)

In [21]:
def get_lane_center_line(lane_id):
    if lane_id in lane_id_2_idx_bike:
        return lanes_bike['center_line'][lane_id_2_idx_bike[lane_id]]
    if lane_id in lane_id_2_idx_not_bike:
        return lanes_not_bike['center_line'][lane_id_2_idx_not_bike[lane_id]]
    else:
        raise ValueError('Lane neither bike only, nor others')
        
def get_lane_len(lane_id):
    return len(get_lane_center_line(lane_id))

lane_2_direct_tl_lights = defaultdict(set)
lane_2_direct_tl_faces = defaultdict(set)
# only lane controlled by traffic light and its predecessor(s) [at least min_required_len_before_tl center line points in total]
min_required_len_before_tl = 30

def get_lane_predecessors(lane_id):
    if lane_id in lane_id_2_idx_bike:
        return lane_adj_list_backward_bike[lane_id_2_idx_bike[lane_id]]
    elif lane_id in lane_id_2_idx_not_bike:
        return lane_adj_list_backward_not_bike[lane_id_2_idx_not_bike[lane_id]]
    else:
        return []
    
def get_lane_successors(lane_id):
    if lane_id in lane_id_2_idx_bike:
        return lane_adj_list_forward_bike[lane_id_2_idx_bike[lane_id]]
    elif lane_id in lane_id_2_idx_not_bike:
        return lane_adj_list_forward_not_bike[lane_id_2_idx_not_bike[lane_id]]
    else:
        return []
    
def get_lane_left_neighbours(lane_id):
    if lane_id in lane_id_2_idx_bike:
        return lane_adj_list_left_bike[lane_id_2_idx_bike[lane_id]]
    elif lane_id in lane_id_2_idx_not_bike:
        return lane_adj_list_left_not_bike[lane_id_2_idx_not_bike[lane_id]]
    else:
        return []
    
def get_lane_right_neighbours(lane_id):
    if lane_id in lane_id_2_idx_bike:
        return lane_adj_list_right_bike[lane_id_2_idx_bike[lane_id]]
    elif lane_id in lane_id_2_idx_not_bike:
        return lane_adj_list_right_not_bike[lane_id_2_idx_not_bike[lane_id]]
    else:
        return []
        
lanes_under_red_turn_fix = [lane_under_red_turn_fix for lane_under_red_turn_fix, _, _, _ in turn_under_red_fixes]

for element in tqdm(proto_API):
    if proto_API.is_lane(element):
        lane_id = MapAPI.id_as_str(element.id)
        if lane_id not in lane_ids_with_wrong_tl_associations:
            lane_traffic_controls = [MapAPI.id_as_str(x) for x in proto_API.get_lane_traffic_controls(lane_id)]
        else:
            lane_traffic_controls = []
        lane_traffic_lights = [x for x in lane_traffic_controls if x in traffic_light_ids_all]
        lane_traffic_faces = [x for x in lane_traffic_controls if x in traffic_faces_ids_all]
        tl_faces_elements = [proto_API[x].element.traffic_control_element for x in lane_traffic_faces]
            
        if lane_id in tl_added_lane_id_2_tl_id:
            lane_traffic_lights.append(tl_added_lane_id_2_tl_id[lane_id])
            lane_traffic_faces.append(tl_added_lane_id_2_tl_id[lane_id])
        if len(lane_traffic_lights):
            dist_from_tl = get_lane_len(lane_id)
            queue = deque() 
            queue.append((lane_id, dist_from_tl))
            while len(queue):
                next_lane, dist_from_tl = queue.popleft()
                lane_2_direct_tl_lights[next_lane].update(lane_traffic_lights)
                if dist_from_tl == get_lane_len(next_lane): # don't propagate faces backwards
                    lane_2_direct_tl_faces[next_lane].update(lane_traffic_faces)
                if dist_from_tl < min_required_len_before_tl and next_lane not in lanes_under_red_turn_fix:
                    lane_predecessors = get_lane_predecessors(next_lane)
                    for lane_predecessor_id in lane_predecessors:
                        queue.append((lane_predecessor_id, dist_from_tl + get_lane_len(lane_predecessor_id)))
                        
# manual sem map fixes
for fix_i, parent_lane in enumerate(lanes_under_red_turn_fix):
    lane_2_direct_tl_lights[f'lane{2*fix_i + 1}'] = lane_2_direct_tl_lights[parent_lane].copy()
    lane_2_direct_tl_faces[f'lane{2*fix_i + 1}'] = lane_2_direct_tl_faces[parent_lane].copy()
    del lane_2_direct_tl_lights[parent_lane]
    del lane_2_direct_tl_faces[parent_lane]
    

tl_light_2_directly_controlled_lanes = defaultdict(set)
for lane_id, tl_lights in lane_2_direct_tl_lights.items():
    for tl_light in tl_lights:
        tl_light_2_directly_controlled_lanes[tl_light].add(lane_id)

In [22]:
master_intersection_idx_2_traffic_lights = []
traffic_light_id_2_master_intersection_idx = defaultdict(list)

for tls_united_by_lane in lane_2_close_tl_controls.values():
    master_intersection_idx = None
    for tl_id in tls_united_by_lane:
        if tl_id in traffic_light_id_2_master_intersection_idx:
            if master_intersection_idx is not None:
                assert traffic_light_id_2_master_intersection_idx[tl_id] == master_intersection_idx, (f'tl_id: {tl_id}', f'traffic_light_id_2_master_intersection_idx[tl_id]: {traffic_light_id_2_master_intersection_idx[tl_id]}', 
                                                                                                      f'master_intersection_idx: {master_intersection_idx}')
            else:
                master_intersection_idx = traffic_light_id_2_master_intersection_idx[tl_id]
            assert tl_id in master_intersection_idx_2_traffic_lights[master_intersection_idx]
        else:
            if master_intersection_idx is None:
                master_intersection_idx = len(master_intersection_idx_2_traffic_lights)
                master_intersection_idx_2_traffic_lights.append(set())
            traffic_light_id_2_master_intersection_idx[tl_id] = master_intersection_idx            
            master_intersection_idx_2_traffic_lights[master_intersection_idx].add(tl_id)
            

In [23]:
len(master_intersection_idx_2_traffic_lights)

9

In [24]:
master_intersection_idx_2_traffic_lights = [set(intersection_tls) for intersection_tls in sorted([tuple(sorted(list(x))) for x in master_intersection_idx_2_traffic_lights])]

In [25]:
if VIS_WIP:
    plt.figure(figsize=(19, 19))

    for master_intersection_i, master_intersection_tlights in enumerate(master_intersection_idx_2_traffic_lights):
        master_x_list, master_y_list = [], []
        for tl_id in master_intersection_tlights:
            tl_coord = traffic_light_id_2_coord[tl_id]
            master_x_list.append(tl_coord[0])
            master_y_list.append(tl_coord[1])
            plt.scatter(tl_coord[0], tl_coord[1], c='b', s=10)
        plt.scatter(np.mean(master_x_list), np.mean(master_y_list), s=70, label=f'{master_intersection_i}')
    plt.legend()

    # for tl_coord in traffic_light_id_2_coord.values():
    #     plt.scatter(tl_coord[0], tl_coord[1], c='black', s=10)


    # # noticed a controlled lane without assigned traffic light
    # lane_id = 'Ib85'
    # lane_center_line = lanes_crosswalks['lanes']['center_line'][lane_id_2_idx[lane_id]]
    # lane_left_board = lanes_crosswalks['lanes']['xy_left_'][lane_id_2_idx[lane_id]]
    # lane_right_board = lanes_crosswalks['lanes']['xy_right_'][lane_id_2_idx[lane_id]]    
    # plt.scatter(lane_center_line[:, 0], lane_center_line[:, 1], s=110)    
    # plt.scatter(lane_left_board[:, 0], lane_left_board[:, 1], s=1, color='grey')    
    # plt.scatter(lane_right_board[:, 0], lane_right_board[:, 1], s=1, color='grey')

    plt.legend()

In [26]:
# splitting the master intersection 1 (there's some long connecting lane likely)
from sklearn.cluster import KMeans
orig_tl_ids = list(master_intersection_idx_2_traffic_lights[3])
coordinates_data = np.array([traffic_light_id_2_coord[tl_id] for tl_id in orig_tl_ids])
kmeans_clusters = KMeans(n_clusters=2, random_state=42).fit(coordinates_data).labels_
master_intersection_idx_new = len(master_intersection_idx_2_traffic_lights)
master_intersection_idx_2_traffic_lights.append(set())
for tl_id, cluster in zip(orig_tl_ids, kmeans_clusters):
    if cluster == 1:
        traffic_light_id_2_master_intersection_idx[tl_id] =  master_intersection_idx_new           
        master_intersection_idx_2_traffic_lights[master_intersection_idx_new].add(tl_id)
        master_intersection_idx_2_traffic_lights[3].remove(tl_id)
        

In [27]:
if VIS_WIP:
    plt.figure(figsize=(19, 19))

    for master_intersection_i, master_intersection_tlights in enumerate(master_intersection_idx_2_traffic_lights):
        master_x_list, master_y_list = [], []
        for tl_id in master_intersection_tlights:
            tl_coord = traffic_light_id_2_coord[tl_id]
            master_x_list.append(tl_coord[0])
            master_y_list.append(tl_coord[1])
            plt.scatter(tl_coord[0], tl_coord[1], c='b', s=10)
        plt.scatter(np.mean(master_x_list), np.mean(master_y_list), s=130, label=f'{master_intersection_i}')
    plt.legend()

Adding missing traffic lights

In [28]:
def get_lane_boarders(lane_id):
    if lane_id in lane_id_2_idx_bike:
        return lanes_bike['xy_left_'][lane_id_2_idx_bike[lane_id]], lanes_bike['xy_right_'][lane_id_2_idx_bike[lane_id]]
    elif lane_id in lane_id_2_idx_not_bike:
        return lanes_not_bike['xy_left_'][lane_id_2_idx_not_bike[lane_id]], lanes_not_bike['xy_right_'][lane_id_2_idx_not_bike[lane_id]]
    else:
        raise ValueError('Lane nether bike only, nor others')
        
def vis_check_unassigned_lanes(master_intersection_i, show_all=False, max_dist=50):
    x_list, y_list, label_list, size_list = [], [], [], []

    master_intersection_tlights = master_intersection_idx_2_traffic_lights[master_intersection_i]
    master_x_list, master_y_list = [], []
    for tl_id in master_intersection_tlights:
        tl_coord = traffic_light_id_2_coord[tl_id]
        master_x_list.append(tl_coord[0])
        master_y_list.append(tl_coord[1])
        x_list.append(tl_coord[0])
        y_list.append(tl_coord[1])
        label_list.append(f'{tl_id}')
        size_list.append(4)
    master_intersection_x, master_intersection_y = np.mean(master_x_list), np.mean(master_y_list)


    for lane_id in list(lane_id_2_idx_bike.keys()) + list(lane_id_2_idx_not_bike.keys()):

        if lane_id not in lane_2_direct_tl_lights or show_all: # there was different check initially, not via lane_2_direct_tl_lights
            lane_center_line = get_lane_center_line(lane_id)
            lane_mass_center = lane_center_line.mean(axis=0)
            if np.hypot(master_intersection_x-lane_mass_center[0], master_intersection_y-lane_mass_center[1]) < max_dist:
                lane_left_board, lane_right_board = get_lane_boarders(lane_id)    
                x_list.extend(lane_center_line[:, 0])
                y_list.extend(lane_center_line[:, 1])
                label_list.extend([f'{lane_id}' for _ in range(len(lane_center_line))])
                size_list.extend(list(np.linspace(0.12, 1.2, len(lane_center_line))))


                x_list.extend(lane_left_board[:, 0])
                y_list.extend(lane_left_board[:, 1])
                label_list.extend(['boarder' for _ in range(len(lane_left_board))])
                size_list.extend([0.1 for _ in range(len(lane_left_board))])


                x_list.extend(lane_right_board[:, 0])
                y_list.extend(lane_right_board[:, 1])
                label_list.extend(['boarder' for _ in range(len(lane_right_board))])
                size_list.extend([0.1 for _ in range(len(lane_right_board))])

    fig = px.scatter(pd.DataFrame({'x': x_list, 'y': y_list, 'label': label_list, 'size': size_list}), 
                     x="x", y="y", color="label", size='size')
    fig.show()
if VIS_WIP:  
    vis_check_unassigned_lanes(3)

In [29]:

# mzvz -491, 896

# F0vz, k0vz -488, 894

# 90Lv -527, 874

# e0Lv,/zLv -530, 876
# LXvy -510, 875
# YVHi -516 901

# TO REMOVE:
# 'LXvy', 'YVHi'

In [30]:
if VIS_WIP:
    vis_check_unassigned_lanes(5)

In [31]:
# jUyh, CVyh -758, 1127
# 57g+ -733, 1125
# W9g+, 38g+, Y8g+ -735, 1122


In [32]:
if VIS_WIP:
    vis_check_unassigned_lanes(0, True, 80)

In [33]:
# T3uS, y3uS -973.5, 1360
# RJu6  -974, 1359
# 8V6+ -950, 1440

# TO REMOVE: 8V6+ (and added a new one, to separate the turn lights)

In [34]:
if VIS_WIP:
    vis_check_unassigned_lanes(9, True)

In [35]:
# RLKQ -35 350

# TO REMOVE:'Bm9O'

In [36]:
if VIS_WIP:
    vis_check_unassigned_lanes(1)

In [37]:
# 8OiE, FHu2 249, 82
# 6PiE 213, 56
# WlSE, pXNd, YkSE,  202, 63

In [38]:
if VIS_WIP:
    vis_check_unassigned_lanes(8)

In [39]:
# wmAx, RRp7 465, -160
# CPtm 439, -190

In [40]:
if VIS_WIP:
    vis_check_unassigned_lanes(2)

In [41]:
# [MapAPI.id_as_str(el) for el in proto_API.get_lane_traffic_controls('i2bp') if MapAPI.id_as_str(el) in traffic_light_ids_all]

In [42]:

# lane_2_direct_tl_lights['RLKQ'] -> {'Luda_13'}, master_intersection_i -> 2

In [43]:
# TO REMOVE: i2bp tl's
# Myw0 684, -407

In [44]:
if VIS_WIP:
    vis_check_unassigned_lanes(4, True)

In [45]:
# '9ie8' (741, -463)
# TO REMOVE: '9ie8'

In [46]:
if VIS_WIP:
    vis_check_unassigned_lanes(7, True)

In [47]:
# due to right turn under red:

# m+dt: , leave connection to li+4; remove connection to Rvxh, lanes to yield on red-light turn
# ['m_dt', 'li+4', ['Rvxh'], ['nwfo', 'AAR1']]

In [48]:
# TnVm, urF8, PrF8  635, -1355
# wqF8 636 -1358
# T1zT, udk+, Pdk+ 672, -1342
# wck+ 667, -1339

# TO REMOVE: AxmM

In [49]:
if VIS_WIP:
    vis_check_unassigned_lanes(6)

In [50]:
# YCLp, H1RI 630, -2242
# m1RI 630, -2242
# ii1t 634, -2268

In [51]:
all_lanes_near_tl = set()
for tl, lanes in tl_control_2_close_lanes.items():
    all_lanes_near_tl.update(lanes)
len(all_lanes_near_tl)

881

In [52]:
def is_the_end_controlled_lane(lane_id, controlled_lanes):
    the_end_lane = True
    for next_lane_id in get_lane_successors(lane_id):
        # small, O(n) is ok
        if next_lane_id in controlled_lanes:
            the_end_lane = False
            break
    return the_end_lane

def is_predecessor(lane_id, candidate_predecessor_lane_id, max_depth=15):
    # bfs
    queue = deque() 
    queue.append((candidate_predecessor_lane_id, 0))
    while len(queue):
        next_lane, depth = queue.popleft()
        if depth + 1 < max_depth: 
            lanes_next = get_lane_successors(next_lane)
            for lane_next_id in lanes_next:
                if lane_next_id == lane_id:
                    return True
                queue.append((lane_next_id, depth + 1))
    return False
    

# the goal is to group traffic light faces controlling the same lanes (assuming they duplicate the same info)
tl_faces_set_2_lanes = defaultdict(set)
tl_faces_set_2_master_intersections = defaultdict(set)
# controlled_lanes_set_2_tl_face_ids = defaultdict(list)

# dummy count for the check only
tl_ids_raw_count = 0
for master_intersection_idx, master_intersection_traffic_lights in enumerate(master_intersection_idx_2_traffic_lights):
    tl_ids_raw_count += len(master_intersection_traffic_lights)
    for tl_id in master_intersection_traffic_lights:
        # all lanes directly controlled by the tl
        controlled_lanes = tl_light_2_directly_controlled_lanes[tl_id]
        
        # differentiate lanes based on tl faces, which are available for the final exit lanes only
        for lane_id in controlled_lanes:
            if is_the_end_controlled_lane(lane_id, controlled_lanes):
                tl_faces_tuple = tuple(sorted(lane_2_direct_tl_faces[lane_id]))
                tl_faces_set_2_lanes[tl_faces_tuple].add(lane_id)
                # propagate the tl-faces set to the predecessors
                for controlled_lane_id in controlled_lanes:
                    if is_predecessor(lane_id, controlled_lane_id):
                        tl_faces_set_2_lanes[tl_faces_tuple].add(controlled_lane_id)
                tl_faces_set_2_master_intersections[tl_faces_tuple].add(master_intersection_idx)

In [53]:
tl_ids_raw_count, len(tl_faces_set_2_lanes)

(120, 63)

In [54]:
for tl_faces, intersection_idx_set in tl_faces_set_2_master_intersections.items():
    assert len(intersection_idx_set) == 1, f'{tl_faces}, {intersection_idx_set}'
    tl_faces_set_2_master_intersections[tl_faces] = list(intersection_idx_set)[0]

In [55]:
# tl_signal_idx corresponds to the set of tl faces with the same set of lanes under control and therefore (presumably) the same signal

controlled_lane_id_2_tl_signal_idx = dict()
exit_lane_id_2_tl_signal_idx = dict()
tl_signal_idx_2_controlled_lanes = []
tl_signal_idx_2_exit_lanes = []
tl_signal_idx_2_stop_coordinates = []
master_intersection_idx_2_tl_signal_indices = defaultdict(list)
tl_face_id_2_tl_signal_indices = defaultdict(set)


min_required_len_after_stop_line = 7

for tl_face_ids, controlled_lanes in sorted(tl_faces_set_2_lanes.items(), key=lambda x: x[0]):
    tl_signal_idx = len(tl_signal_idx_2_controlled_lanes)
    for lane_id in controlled_lanes:
        controlled_lane_id_2_tl_signal_idx[lane_id] = tl_signal_idx 
    tl_signal_idx_2_controlled_lanes.append(set(controlled_lanes))
    master_intersection_idx_2_tl_signal_indices[tl_faces_set_2_master_intersections[tl_face_ids]].append(tl_signal_idx)
    stop_coordinates = []
    tl_signal_idx_2_exit_lanes.append(set())
    for lane_id in controlled_lanes:
        
        if is_the_end_controlled_lane(lane_id, controlled_lanes):
            stop_coordinates.append(get_lane_center_line(lane_id)[-1])
            
            dist_from_stop_line = 0
            queue = deque() 
            queue.append((lane_id, dist_from_stop_line))
            while len(queue):
                next_lane, dist_from_stop_line = queue.popleft()
                if dist_from_stop_line != 0: # not the end lane
                    tl_signal_idx_2_exit_lanes[tl_signal_idx].add(next_lane)
                    exit_lane_id_2_tl_signal_idx[next_lane] = tl_signal_idx
                if dist_from_stop_line < min_required_len_after_stop_line:
                    lanes_next = get_lane_successors(next_lane)
                    for lane_next_id in lanes_next:
                        queue.append((lane_next_id, dist_from_stop_line + get_lane_len(lane_next_id)))
                
                
    tl_signal_idx_2_stop_coordinates.append(stop_coordinates)
    for tl_face_id in tl_face_ids:
        tl_face_id_2_tl_signal_indices[tl_face_id].add(tl_signal_idx)

In [56]:
# for tl_face_id, tl_signals in tl_face_id_2_tl_signal_indices.items():
#     assert len(tl_signals) == 1, f'{tl_face_id}, {tl_signals}'

--------------------
vis check

In [57]:
def vis_tl_signal(tl_signal_indices, title, with_boarders=True, show_lane_ids=False):
    x_list, y_list, label_list, size_list = [], [], [], []

    for tl_signal_idx in tl_signal_indices:
        for lane_id in tl_signal_idx_2_controlled_lanes[tl_signal_idx]:
            lane_center_line = get_lane_center_line(lane_id)
            lane_mass_center = lane_center_line.mean(axis=0)
            lane_left_board, lane_right_board = get_lane_boarders(lane_id)   
            x_list.extend(lane_center_line[:, 0])
            y_list.extend(lane_center_line[:, 1])
            label_str = f'before {tl_signal_idx}'
            if show_lane_ids:
                label_str += f': {lane_id}'
            label_list.extend([label_str for _ in range(len(lane_center_line))])
            size_list.extend(list(np.linspace(0.12, 1.2, len(lane_center_line))))


            if with_boarders:
                x_list.extend(lane_left_board[:, 0])
                y_list.extend(lane_left_board[:, 1])
                label_list.extend([f'boarder {tl_signal_idx}' for _ in range(len(lane_left_board))])
                size_list.extend([0.1 for _ in range(len(lane_left_board))])

                x_list.extend(lane_right_board[:, 0])
                y_list.extend(lane_right_board[:, 1])
                label_list.extend([f'boarder {tl_signal_idx}' for _ in range(len(lane_right_board))])
                size_list.extend([0.1 for _ in range(len(lane_right_board))])

        for lane_id in tl_signal_idx_2_exit_lanes[tl_signal_idx]:
            lane_center_line = get_lane_center_line(lane_id)
            lane_mass_center = lane_center_line.mean(axis=0)
            lane_left_board, lane_right_board = get_lane_boarders(lane_id)  
            x_list.extend(lane_center_line[:, 0])
            y_list.extend(lane_center_line[:, 1])
            label_str = f'exit {tl_signal_idx}'
            if show_lane_ids:
                label_str += f': {lane_id}'
            label_list.extend([label_str for _ in range(len(lane_center_line))])
            size_list.extend(list(np.linspace(0.12, 1.2, len(lane_center_line))))

            
            if with_boarders:
                x_list.extend(lane_left_board[:, 0])
                y_list.extend(lane_left_board[:, 1])
                label_list.extend([f'boarder {tl_signal_idx}' for _ in range(len(lane_left_board))])
                size_list.extend([0.1 for _ in range(len(lane_left_board))])

                x_list.extend(lane_right_board[:, 0])
                y_list.extend(lane_right_board[:, 1])
                label_list.extend([f'boarder {tl_signal_idx}' for _ in range(len(lane_right_board))])
                size_list.extend([0.1 for _ in range(len(lane_right_board))])

        for stop_point in tl_signal_idx_2_stop_coordinates[tl_signal_idx]:
            x_list.append(stop_point[0])
            y_list.append(stop_point[1])
            label_list.append(f'STOP {tl_signal_idx}')
            size_list.append(2)
        

    fig = px.scatter(pd.DataFrame({'x': x_list, 'y': y_list, 'label': label_list, 'size': size_list}), 
                     x="x", y="y", color="label", size='size', 
                    title=title)
    fig.show()

if VIS_WIP:
    for master_intersection_idx in [7]: #range(len(master_intersection_idx_2_tl_signal_indices)):
        for tl_signal_idx in master_intersection_idx_2_tl_signal_indices[master_intersection_idx]:
            vis_tl_signal([tl_signal_idx], f'Master intersection {master_intersection_idx}: {tl_signal_idx}', with_boarders=False, show_lane_ids=True)

In [58]:
intersection_id = 6
if VIS_WIP:
    vis_tl_signal(master_intersection_idx_2_tl_signal_indices[intersection_id], f'Master intersection {intersection_id}', with_boarders=False, show_lane_ids=True)

In [59]:
get_lane_predecessors('LGhV'), get_lane_predecessors('8V6+') 

(['8V6+'], ['lXo5'])

In [60]:
lane_2_direct_tl_lights['Bm9O']

set()

In [61]:
get_lane_successors('9ie8') 
# '/zLv', ['Pl2o'], '3KTc', ['4LEI']
# 'VUqQ', ['YeLI'], 'fMAY', ['/9cr', 'KtDh']
# 'cKUZ', ['x+C2'], 'b2Km', ['mTBO', 'JDFg']
# 'Myw0', ['ZjWH', '/9cr', 'biWH', '6iWH'], 'KHxd', ['x+C2', 'SE96']
# '89HH', ['RbC9'], '5z+h', ['wqVO', 'yU+6']
# 'jMm1', ['KTUI'], 'wTjJ', ['SOK7', 'JhJI', 'UCr8']
# 'YkSE', ['SOK7'], '+Klg', ['RbC9', 'JAlK']

# 'ake8', ['z5yd'], '+SHD', ['xb0W', '/aG3', 'A+L+']
# 'vijX', ['A+L+'], 'H09x', ['u016', 'wc+d']
# 'hINg', ['2OcN', 'XOcN'], 'VPcN', ['z5yd', '8weJ']
# 'aVLS', ['zojk'], '+B39', ['I4aR']
# 'CVyh', ['1WGF', 'WWGF', 'wtG4', 'PuG4'], 'UXGF', ['zojk']
# 'emZY', ['84vV'], 'FWhR', []
# 'YCLp', ['zvWg'], 'L5L1', ['84vV']
# 'TnVm', ['nwfo'], 'hQc1', ['78bQ', 'M3Fu']
# 'DvGy', ['qgZI'], 'UB+4', ['+P/J', '9qCn']
# 'b75w', ['SYA4'], '4gKt', ['Q20G']

['pQYu']

In [62]:
# '/E65', ['u016'], 'Waxo', ['qSSB', 'pQYu']
# 'T1zT' ['nicu'], 'hE9x', ['li+4']
# 'RRp7', ['9qCn'], 'le35', ['SYA4']

-----------------

In [63]:
lane_2_master_intersection_related_lanes = dict() #->set
lane_id_2_master_intersection_idx = dict()
for master_intersection_idx, master_intersection_tl_signal_indices in master_intersection_idx_2_tl_signal_indices.items():
    all_master_intersection_lanes = set()
    for tl_signal_idx in master_intersection_tl_signal_indices:
        all_master_intersection_lanes.update(tl_signal_idx_2_controlled_lanes[tl_signal_idx])
        all_master_intersection_lanes.update(tl_signal_idx_2_exit_lanes[tl_signal_idx])
        for lane_id in tl_signal_idx_2_controlled_lanes[tl_signal_idx].union(tl_signal_idx_2_exit_lanes[tl_signal_idx]):
            lane_id_2_master_intersection_idx[lane_id] = master_intersection_idx
        
        # to also associate the neighbourhood to an intersection, bfs from the controlled lanes (mainly to include the artificially separated turns-under-red)
        lanes_closed_set = tl_signal_idx_2_controlled_lanes[tl_signal_idx].union(tl_signal_idx_2_exit_lanes[tl_signal_idx])
        for lane_id in tl_signal_idx_2_controlled_lanes[tl_signal_idx]:
            dist_from_exit = 0
            queue = deque() 
            queue.append((lane_id, dist_from_exit))
            while len(queue):
                next_lane_id, dist_from_exit = queue.popleft()
                lane_id_2_master_intersection_idx[next_lane_id] = master_intersection_idx
                if dist_from_exit < 70: # points
                    lanes_next = get_lane_successors(next_lane_id) + get_lane_left_neighbours(next_lane_id) + get_lane_right_neighbours(next_lane_id)
                    for further_lane_id in lanes_next:
                        if further_lane_id not in lanes_closed_set:
                            queue.append((further_lane_id, dist_from_exit + get_lane_len(further_lane_id)))
                            lanes_closed_set.add(further_lane_id)
    for lane_id in all_master_intersection_lanes:
        lane_2_master_intersection_related_lanes[lane_id] = {neighbour_lane_id for neighbour_lane_id in all_master_intersection_lanes 
                                                             if neighbour_lane_id != lane_id}

In [64]:
# added lanes belong to intersections
lanes_not_bike_intersection, lane_id_2_idx_not_bike_intersection = get_lanes_dict_and_id_mapping(filter_function=lambda x: x in added_lanes or (not proto_API.is_bike_only_lane(x) and x in lane_id_2_master_intersection_idx))
lanes_bike_intersection, lane_id_2_idx_bike_intersection = get_lanes_dict_and_id_mapping(filter_function=lambda x: x in added_lanes or (proto_API.is_bike_only_lane(x) and x in lane_id_2_master_intersection_idx))
lanes_intersection, lane_id_2_idx_intersection = get_lanes_dict_and_id_mapping(filter_function=lambda x: x in added_lanes or x in lane_id_2_master_intersection_idx)

kd_tree_bike_intersection, kd_idx_2_lane_id_idx_bike_intersection = get_kd_tree_and_idx_map(lanes_bike_intersection)
kd_tree_not_bike_intersection, kd_idx_2_lane_id_idx_not_bike_intersection = get_kd_tree_and_idx_map(lanes_not_bike_intersection)
kd_tree_intersection, kd_idx_2_lane_id_idx_intersection = get_kd_tree_and_idx_map(lanes_intersection)

-----------------------------------------------------

In [65]:
# problematic_tl_faces = []
# for tl_id, face_id in tqdm(eval_zarr.tl_faces[('traffic_light_id', 'face_id')], total=len(eval_zarr.tl_faces)):
#     if tl_id == 'Ae1d':
#         problematic_tl_faces.append(face_id)

In [66]:
# agent_velocity_99perc = np.percentile(eval_zarr.agents['velocity'], 99)
# agent_velocity_99perc
# # 12.295

In [67]:
# traffic_faces_ids_all = set(eval_zarr.tl_faces['face_id'])
# len(traffic_light_ids_all), len(traffic_faces_ids_all)
# #(99, 297)

In [68]:
# len(traffic_light_ids_all.intersection(set(tl_control_2_close_lanes.keys())))/len(traffic_light_ids_all), len(traffic_faces_ids_all.intersection(set(tl_control_2_close_lanes.keys())))/len(traffic_faces_ids_all)
# # (0.9494949494949495, 0.0)

In [69]:
def get_traffic_light_coordinates(tl_el_id):
    if el_id in proto_API.ids_to_el:
        coordinates = proto_API.get_traffic_light_coords(el_id)['xyz']
        return coordinates[:, :2].mean(axis=0)

----------------
visual check

In [70]:
min([len(x) for x in lanes_crosswalks['lanes']['center_line']])

1

In [71]:
def get_closest_lanes(coord, agent_class, k_nearest, intersections_only=False):
    if agent_class == BIKE_CLASS:
        if intersections_only:
            return kd_tree_bike_intersection.query(coord, k=k_nearest)
        return kd_tree_bike.query(coord, k=k_nearest)
    elif agent_class == CAR_CLASS:
        if intersections_only:
            return kd_tree_not_bike_intersection.query(coord, k=k_nearest)
        return kd_tree_not_bike.query(coord, k=k_nearest)
    elif agent_class == ALL_WHEELS_CLASS:
        if intersections_only:
            return kd_tree_intersection.query(coord, k=k_nearest)
    else:
        raise NotImplementedError('Only bikes and general cars supported')
        
def get_lane_point_from_kdtree(candidate_idx, agent_class, intersections_only=False):
    if agent_class == BIKE_CLASS:
        if intersections_only:
            return kd_idx_2_lane_id_idx_bike_intersection[candidate_idx]
        return kd_idx_2_lane_id_idx_bike[candidate_idx]
    elif agent_class == CAR_CLASS:
        if intersections_only:
            return kd_idx_2_lane_id_idx_not_bike_intersection[candidate_idx]
        return kd_idx_2_lane_id_idx_not_bike[candidate_idx]
    elif agent_class == ALL_WHEELS_CLASS:
        if intersections_only:
            return kd_idx_2_lane_id_idx_intersection[candidate_idx]
        return kd_idx_2_lane_id_idx[candidate_idx]

In [72]:
def cos_dist(x1, y1, yaw_rad):
    x2 = np.cos(yaw_rad)
    y2 = np.sin(yaw_rad)
    vector_product = x1 * x2 + y1 * y2
    cos = vector_product / (np.hypot(x1, y1) * np.hypot(x2, y2))
    return 1 - cos

def is_end_of_maneuver_lane(point_idx, lane_len, overlapped_last_points_count=5, min_len_maneuver_lane=7):
    return lane_len > min_len_maneuver_lane and point_idx >= lane_len - 1 - overlapped_last_points_count

def find_closest_lane(coord, yaw, agent_class, lane_point_2_blocked_lanes_set=None,
                      max_dist_m=4, k_nearest=25, return_point_i=False, 
                      return_blocked_tl_signals=False, blocked_dist_threshold_m=1.5, 
                      too_close_insensitivity_m=1.3, min_cos_dist_threshold=np.pi/2,
                      intersections_only=False, vis=False):
    if vis: 
        plt.figure(figsize=(15, 15))
    candidate_distances, candidate_indices = get_closest_lanes(coord, agent_class, k_nearest, intersections_only)    
    
    closed_set = set()
    min_cos_dist = float('inf')
    min_dist = candidate_distances[0]
    closest_lane_current_i = None
    result = None
    
    # processing in increasing order of L2
    def is_best_dist(lane_cos_dist, min_cos_dist, dist):
        if dist == min_dist:
            deadband = 0
        elif dist <= too_close_insensitivity_m:
            deadband = 0.001
        else:
            deadband = 0.05
        return lane_cos_dist + deadband < min_cos_dist
    
    for i, candidate_idx in enumerate(candidate_indices):
        lane_id, point_idx = get_lane_point_from_kdtree(candidate_idx, agent_class, intersections_only)
        if candidate_distances[i] > max_dist_m:
            break
        if lane_id not in closed_set:
            closed_set.add(lane_id)
            lane_center_line = get_lane_center_line(lane_id)
            lane_cos_dists =[]
            if point_idx > 0:
                x1_prev = lane_center_line[point_idx][0] - lane_center_line[point_idx - 1][0]
                y1_prev = lane_center_line[point_idx][1] - lane_center_line[point_idx - 1][1] 
                lane_cos_dists.append(cos_dist(x1_prev, y1_prev, yaw))
            if point_idx + 1 < len(lane_center_line):
                x1_next = lane_center_line[point_idx + 1][0] - lane_center_line[point_idx][0]
                y1_next = lane_center_line[point_idx + 1][1] - lane_center_line[point_idx][1] 
                lane_cos_dists.append(cos_dist(x1_next, y1_next, yaw))   
                
            lane_cos_dist = np.mean(lane_cos_dists)
            
            if vis: 
                plt.scatter(lane_center_line[:, 0], lane_center_line[:, 1], label=f'{lane_id} ({lane_cos_dist:.6f} ({is_best_dist(lane_cos_dist, min_cos_dist, candidate_distances[i])}) ({lane_cos_dists}), {candidate_distances[i]:.2f} m')
#             if is_end_of_maneuver_lane(point_idx, len(lane_center_line)):
#                 # end-of-line candidates are lower priority
#                 if point_start > 0:
#                     x1_prev = lane_center_line[point_start][0] - lane_center_line[point_start - 1][0]
#                     y1_prev = lane_center_line[point_start][1] - lane_center_line[point_start - 1][1] 
#                     lane_cos_dist = (lane_cos_dist + cos_dist(x1_prev, y1_prev, yaw))/2
#                 if is_best_cos_dist(lane_cos_dist, min_cos_dist_end_of_line_leage):
#                     min_cos_dist_end_of_line_leage = lane_cos_dist
#                     closest_lane_current_i_end_of_line_leage = i
#                     if return_point_i:
#                         result_end_of_line_leage = (lane_id, point_idx)
#                     else:
#                         result_end_of_line_leage = lane_id
    
#             else:
            if is_best_dist(lane_cos_dist, min_cos_dist, candidate_distances[i]):
                min_cos_dist = lane_cos_dist
                closest_lane_current_i = i
                result = (lane_id, point_idx)
    if vis: 
        plt.scatter(coord[0], coord[1], c='yellow', s=30)
        plt.arrow(coord[0], coord[1], 3*np.cos(yaw), 3*np.sin(yaw), color='r')
        plt.legend()
        plt.title(f'{result}, {lane_point_2_blocked_lanes_set.get(result, set()) if lane_point_2_blocked_lanes_set is not None else ""}')

    if result is None or min_cos_dist >= min_cos_dist_threshold:
        return None
    closest_lane = result[0]
                    
    if return_blocked_tl_signals:
        if lane_point_2_blocked_lanes_set is None:
            last_blocked_i = closest_lane_current_i
            while (last_blocked_i < len(candidate_distances) and 
                   candidate_distances[last_blocked_i] < candidate_distances[closest_lane_current_i] + blocked_dist_threshold_m):
                last_blocked_i += 1
            blocked_lane_candidates = [get_lane_point_from_kdtree(idx, agent_class, intersections_only) for idx in candidate_indices[:last_blocked_i]]

            blocked_lane_candidates = [lane_id for lane_id, lane_point_i in blocked_lane_candidates
                                      if not is_end_of_maneuver_lane(lane_point_i, get_lane_len(lane_id))]
            # to avoid blocking its own tl:
            if closest_lane in exit_lane_id_2_tl_signal_idx:
                tl_signal_idx_closest_lane = exit_lane_id_2_tl_signal_idx[closest_lane] 
                blocked_signals = {exit_lane_id_2_tl_signal_idx[lane_id_] for lane_id_ in blocked_lane_candidates 
                                   if (lane_id_ in exit_lane_id_2_tl_signal_idx 
                                       and exit_lane_id_2_tl_signal_idx[lane_id_] != tl_signal_idx_closest_lane)}
            else:
                blocked_signals = set()
        else:
            blocked_signals = lane_point_2_blocked_lanes_set.get(result, set())
        if not return_point_i:
            return closest_lane, blocked_signals
        return result, blocked_signals
    if not return_point_i:
        return closest_lane 
    return result

In [73]:
lane_point_2_blocked_lanes_set = dict()

checked_hard_cases = {('csUz', 2), ('RmBi', 7), ('RmBi', 6)}

for i, lane_id in tqdm(enumerate(exit_lane_id_2_tl_signal_idx.keys()), desc='Blocked sets..'):
#     if i < 116: continue
    lane_len = get_lane_len(lane_id)
    lane_center_line = get_lane_center_line(lane_id)
    assert lane_len == len(lane_center_line)
    blocked_set_so_far = set()
    for point_idx in range(lane_len - 1, -1, -1):  
        yaws =[]
        if point_idx > 0:
            x1_prev = lane_center_line[point_idx][0] - lane_center_line[point_idx - 1][0]
            y1_prev = lane_center_line[point_idx][1] - lane_center_line[point_idx - 1][1] 
            yaws.append(get_helping_angle(np.array([x1_prev, y1_prev])))
        if point_idx + 1 < lane_len:
            x1_next = lane_center_line[point_idx + 1][0] - lane_center_line[point_idx][0]
            y1_next = lane_center_line[point_idx + 1][1] - lane_center_line[point_idx][1] 
            yaws.append(get_helping_angle(np.array([x1_next, y1_next])))
        # 1st and 4th quadrants boarder
        yaws = sorted(yaws)
#         print('yaws init', yaws)
        if len(yaws) > 1 and yaws[0] < np.pi/2 and yaws[1] > 3*np.pi/2:
            yaws[1] = yaws[1] - 2*np.pi
        yaw = np.mean(yaws)
#         print(yaws, [[x1_prev, y1_prev], [x1_next, y1_next]])
        
        (closest_lane_id, closest_lane_point_i), blocked_tl_signals = find_closest_lane(lane_center_line[point_idx], yaw, 
                                                                                          agent_class=ALL_WHEELS_CLASS, 
                                                                                          k_nearest=15, return_point_i=True, 
                                                                                          return_blocked_tl_signals=True, 
                                                                                       intersections_only=True)
        assert (lane_id, point_idx) in checked_hard_cases or closest_lane_id == lane_id and point_idx == closest_lane_point_i, f'closest_lane_id: {closest_lane_id}[{closest_lane_point_i}], (true lane_id: {lane_id}[{point_idx}])'
        blocked_set_so_far.update(blocked_tl_signals)  
        lane_point_2_blocked_lanes_set[(lane_id, point_idx)] = deepcopy(blocked_set_so_far)

In [74]:
def get_info_per_related_lanes(frame_sample):
    timestamp = datetime.fromtimestamp(frame_sample['timestamp']/10**9).astimezone(timezone('US/Pacific'))
    ego_centroid = frame_sample['ego_centroid']
    ego_yaw = frame_sample['ego_yaw']
    ego_closest_lane_id = find_closest_lane(ego_centroid, ego_yaw, agent_class=ALL_WHEELS_CLASS, intersections_only=True)
    intersection_related_lanes = lane_2_master_intersection_related_lanes[ego_closest_lane_id] if ego_closest_lane_id in lane_2_master_intersection_related_lanes else set()
    if len(intersection_related_lanes) == 0:
        return []
    lane_2_speeds = defaultdict(list)
    lane_2_completions = defaultdict(list)
    lane_2_blocked_tl_signals = defaultdict(set)
    agents_with_wheels = [(agent, ALL_WHEELS_CLASS if np.nonzero(agent[-1])[0][0] == CAR_CLASS else BIKE_CLASS) for agent in frame_sample['agents'] if np.nonzero(agent[-1])[0][0] in [CAR_CLASS, BIKE_CLASS]]
    ego_speed = frame_sample['ego_speed']
    if ego_speed is not None:
        agents_with_wheels.append(((ego_centroid, None, ego_yaw, ego_speed, None, None), ALL_WHEELS_CLASS))
        intersection_related_lanes.add(ego_closest_lane_id)
    for agent, agent_class in agents_with_wheels:
        agent_speed = np.hypot(*agent[-3])
        agent_centroid = agent[0]
        agent_yaw = agent[2]
        find_closest_result = find_closest_lane(agent_centroid, agent_yaw, agent_class,
                                                                    lane_point_2_blocked_lanes_set=lane_point_2_blocked_lanes_set,
                                                                    return_point_i=True, 
                                                                    return_blocked_tl_signals=True, 
                                                                    intersections_only=True)
        if find_closest_result is not None:
            (lane_id, lane_point_i), blocked_tl_signals = find_closest_result
            lane_len = get_lane_len(lane_id)
            if not is_end_of_maneuver_lane(lane_point_i, lane_len):       
                lane_completion = lane_point_i/(lane_len - 1)
                if lane_id in intersection_related_lanes:
                    lane_2_speeds[lane_id].append(agent_speed)
                    lane_2_completions[lane_id].append(lane_completion)
                    lane_2_blocked_tl_signals[lane_id].update(blocked_tl_signals)
    lane_results = []
    for lane_id, speeds in lane_2_speeds.items():
        lane_results.append((lane_id, np.mean(speeds), np.max(speeds), len(speeds), np.min(lane_2_completions[lane_id]), lane_2_blocked_tl_signals[lane_id]))
    
    # tl's
    tl_results = set()
    tl_faces = filter_tl_faces_by_status(frame_sample['tl_faces'], "ACTIVE")
    for tl_face_id, tl_light_id, _ in tl_faces:
        tl_color = proto_API.get_traffic_face_colour(tl_face_id)
        if tl_color != 'unknown':
            for tl_signal_idx in tl_face_id_2_tl_signal_indices[tl_face_id]:
                for tl_controlled_lane in tl_signal_idx_2_controlled_lanes[tl_signal_idx]:
                    if tl_controlled_lane in intersection_related_lanes:
                        tl_color_code = TL_GREEN_COLOR if tl_color == 'green' else TL_RED_COLOR if tl_color == 'red' else TL_YELLOW_COLOR
                        tl_results.add((tl_light_id, tl_signal_idx, tl_color_code))
    
    if len(lane_results) or len(tl_results):
        master_intersection_idx = lane_id_2_master_intersection_idx[ego_closest_lane_id]
        results = (master_intersection_idx, timestamp, ego_centroid, ego_yaw, lane_results, tl_results)
    else:
        results = []
    return results

def get_tl_signal_current(lanes_info_only, tl_faces_info,
                          speed_activation_threshold=1.5, 
                          is_close_to_lane_start_completion_threshold=0.45, 
                          lane_stopped_speed_threshold=0.1, 
                          stopped_cars_min_count=2
                         ):
    tl_signals_GO, tl_signals_STOP, tl_events = set(), set(), []
    
    if len(lanes_info_only) == 0:
        return tl_signals_GO, tl_signals_STOP, tl_events   
    
    moving_lane_current_indices, stopped_lane_current_indices = [], []
    for current_i, lane_record in enumerate(lanes_info_only):
        if lane_record[1] < speed_activation_threshold:
            stopped_lane_current_indices.append(current_i)
        else:
            moving_lane_current_indices.append(current_i)
            
    # a car not moving doesn't sufficiently mean STOP for a line, but it's a suspect
    tl_signals_STOP_suspects = set()
    signal_2_stopped_car_speed_count = dict()
    signal_2_stopped_car_lanes = defaultdict(list)
    tl_stop_candidate_2_some_responsible_lane = dict()
    for current_i in stopped_lane_current_indices:
        lane_id, _, speed_max, car_counts, _, _ = lanes_info_only[current_i]
        if lane_id in controlled_lane_id_2_tl_signal_idx:
            controlling_tl_signal = controlled_lane_id_2_tl_signal_idx[lane_id]
            prev_speed, prev_count = signal_2_stopped_car_speed_count.get(controlling_tl_signal, (0, 0))
            signal_2_stopped_car_lanes[controlling_tl_signal].append(lane_id)
            signal_2_stopped_car_speed_count[controlling_tl_signal] = (max(prev_speed, speed_max), prev_count + car_counts)
            if (speed_max < lane_stopped_speed_threshold and 
                car_counts >= stopped_cars_min_count):
                    tl_signals_STOP_suspects.add(controlling_tl_signal)
                    tl_stop_candidate_2_some_responsible_lane[controlling_tl_signal] = lane_id
    # for the case of too short consec lanes
    for controlling_tl_signal, speed_count in signal_2_stopped_car_speed_count.items():
        speed_max, car_counts = speed_count
        if (speed_max < lane_stopped_speed_threshold and 
                car_counts >= stopped_cars_min_count):
                    tl_signals_STOP_suspects.add(controlling_tl_signal)
                    tl_stop_candidate_2_some_responsible_lane[controlling_tl_signal] = '_'.join(sorted(signal_2_stopped_car_lanes[controlling_tl_signal]))
            
    for current_i in moving_lane_current_indices:
        # tl exit lanes with moving cars close to start suggest status GO for corresponding tl
        lane_id, _, speed_max, _, min_lane_completion, blocked_tl_signals = lanes_info_only[current_i]
        if (min_lane_completion < is_close_to_lane_start_completion_threshold and
            min_lane_completion > 0 and
            lane_id in exit_lane_id_2_tl_signal_idx):
            tl_signal_idx = exit_lane_id_2_tl_signal_idx[lane_id]
            tl_signals_GO.add(tl_signal_idx)
            tl_events.append((lane_id, tl_signal_idx, TL_GREEN_COLOR))
            tl_signals_STOP.update(blocked_tl_signals)
            for tl_signal_idx in blocked_tl_signals:
                tl_events.append((f'{lane_id}_block', tl_signal_idx, TL_RED_COLOR))
        if (lane_id in controlled_lane_id_2_tl_signal_idx and 
            speed_max > lane_stopped_speed_threshold and
            controlled_lane_id_2_tl_signal_idx[lane_id] in tl_signals_STOP_suspects
           ):
            tl_signals_STOP_suspects.remove(controlled_lane_id_2_tl_signal_idx[lane_id])
        
        
    remaining_suspects = tl_signals_STOP_suspects.difference(tl_signals_GO)
    tl_signals_STOP.update(remaining_suspects)
    for tl_signal_idx in remaining_suspects:
        tl_events.append((tl_stop_candidate_2_some_responsible_lane[tl_signal_idx], tl_signal_idx, TL_RED_COLOR))
        
    # processing observable tl_faces
    for tl_light_id, tl_signal_id, color_code in tl_faces_info:
        if color_code == TL_RED_COLOR:
            tl_signals_STOP.add(tl_signal_id)
        elif color_code == TL_GREEN_COLOR:
            tl_signals_GO.add(tl_signal_id)
        tl_events.append((tl_light_id, tl_signal_id, color_code))
    return tl_signals_GO.difference(tl_signals_STOP), tl_signals_STOP, set(tl_events)
            
        
def get_accumulated_tl_signals(timestamp, ego_centroid, master_intersection_idx,
                               tl_signals_GO, tl_signals_STOP, 
                               timestamp_prev, ego_centroid_prev, master_intersection_idx_prev,
                               tl_signals_buffer,
                               timediff_max_sec=2, distdiff_max_m=2, max_buffer_age_sec=15
                              ):
    timediff_sec = (timestamp - timestamp_prev).total_seconds()
    if master_intersection_idx != master_intersection_idx_prev:
        tl_signals_buffer = dict()
    elif timediff_sec > timediff_max_sec:
        tl_signals_buffer = dict()
    else:
        translation_m = np.hypot(ego_centroid_prev[0] - ego_centroid[0], ego_centroid_prev[1] - ego_centroid[1])
        if translation_m > distdiff_max_m:
            tl_signals_buffer = dict()
    for tl_signal_green_idx in tl_signals_GO:
        tl_signals_buffer[tl_signal_green_idx] = (timestamp, 1)
    for tl_signal_red_idx in tl_signals_STOP:
        tl_signals_buffer[tl_signal_red_idx] = (timestamp, 0)
        
    current_buffer_items = list(tl_signals_buffer.items())
    for tl_sig_idx, (timestamp_record, color_cod) in current_buffer_items:
        if (timestamp - timestamp_record).total_seconds() > max_buffer_age_sec:
            del tl_signals_buffer[tl_sig_idx]
    return tl_signals_buffer

In [75]:
# snapshot = tracemalloc.take_snapshot()
# display_top(snapshot)

In [103]:
import matplotlib.pyplot as plt

import numpy as np

from l5kit.data import ChunkedDataset, LocalDataManager
from l5kit.dataset import EgoDataset, AgentDataset

from l5kit.rasterization import build_rasterizer
from l5kit.configs import load_config_data
from l5kit.visualization import draw_trajectory, TARGET_POINTS_COLOR
from l5kit.geometry import transform_points
from IPython.display import display, clear_output
import PIL
import time

import cv2
from tqdm.auto import tqdm

import os

os.environ["L5KIT_DATA_FOLDER"] = "../input/"
# get config
cfg = load_config_data("./visualisation_config.yaml")
cfg['raster_params']['raster_size'] = [350, 350] #[532, 532]
# cfg['raster_params']['pixel_size'] = [1, 1]
# cfg['raster_params']['ego_center'] = [0.4, 0.5]

dm = LocalDataManager()
dataset_path = dm.require('scenes/train_full_filtered_min_frame_history_4_min_frame_future_1.zarr')#validate_filtered_min_frame_history_4_min_frame_future_1.zarr')
zarr_dataset_filtered = ChunkedDataset(dataset_path)
zarr_dataset_filtered.open()

cfg["raster_params"]["map_type"] = "py_semantic"
rast = build_rasterizer(cfg, dm)
dataset_filtered = EgoDataset(cfg, zarr_dataset_filtered, rast)

frame_dataset = FramesDataset(zarr_dataset_filtered)

In [101]:

cycled_colors = plt.get_cmap('tab20b')(np.linspace(0, 1, 20))
cycled_colors_targets = plt.get_cmap('Set2')(np.linspace(0, 1, 8))

def plot_line(ax, line_id, speed=None, completion=None, speed_min=0, speed_max=12.295, color=None, text1='', text2=''): #99th speed percentile
    lane_center_line = get_lane_center_line(line_id)
    lane_left_board, lane_right_board = get_lane_boarders(line_id)    
    if speed is None:
        ax.scatter(lane_center_line[:, 0], lane_center_line[:, 1], s=11, color=color) 
        ax.text(lane_center_line[:, 0].mean() + 1.7, lane_center_line[:, 1].mean() + 4, 
                text1, 
                fontsize=14)
        ax.text(lane_center_line[:, 0].mean() - 12, lane_center_line[:, 1].mean() - 1.7, 
                text2, color=color,
                fontsize=12)
#                     ax.scatter(lane_left_board[:, 0], lane_left_board[:, 1], s=1, color='grey')    
#                     ax.scatter(lane_right_board[:, 0], lane_right_board[:, 1], s=1, color='grey')
    else:
        speed = np.clip(speed, speed_min, speed_max)
        color = plt.get_cmap('Oranges')(speed/speed_max)
        completion_idx = int((len(lane_center_line) - 1)*completion)
        ax.scatter(lane_center_line[:, 0], lane_center_line[:, 1], s=0.2, color=color)    
#                     ax.scatter(lane_left_board[:completion_idx, 0], lane_left_board[:completion_idx, 1], s=0.75, color=color)    
#                     ax.scatter(lane_right_board[:completion_idx, 0], lane_right_board[:completion_idx, 1], s=0.75, color=color)


        ax.scatter(lane_center_line[completion_idx, 0], lane_center_line[completion_idx, 1], s=3, color=color) 
        speed_activation_threshold=1.5
        ax.text(lane_center_line[completion_idx, 0], lane_center_line[completion_idx, 1] + 0.7, 
                f'{speed:.0f} ({line_id})', color=color if speed > speed_activation_threshold else 'grey', 
                fontsize=14)
        if completion_idx + 1 < len(lane_center_line):
            ax.scatter(lane_center_line[completion_idx + 1, 0], lane_center_line[completion_idx + 1, 1], s=12, color=color) 
#                     ax.scatter(lane_left_board[completion_idx:, 0], lane_left_board[completion_idx:, 1], s=0.1, color=color)    
#                     ax.scatter(lane_right_board[completion_idx:, 0], lane_right_board[completion_idx:, 1], s=0.1, color=color)

    
    
def store_output(im, output, start_scene, end_scene, intersection, show):
    if output is None:
        start_scene = f'{"".join(["0" for _ in range(3 - len(str(start_scene)))])}{start_scene}'
        end_scene = f'{"".join(["0" for _ in range(3 - len(str(end_scene)))])}{end_scene}'
        output = cv2.VideoWriter(f'../outputs/validate_vis_check_{str(intersection) + "_" if intersection is not None else ""}scenes_{start_scene}_{end_scene}_full.avi',
                                 cv2.VideoWriter_fourcc('M', 'J', 'P', 'G'), 10, (im.shape[1], im.shape[0]))
    output.write(cv2.cvtColor(im, cv2.COLOR_RGB2BGR))
    if show:
        clear_output(wait=True)
        display(PIL.Image.fromarray(im))
        time.sleep(0.01)
    return output

def get_rnn_inputs_from_events(obsereved_events):
    obsereved_events = [x for x in obsereved_events if '_block' not in x[0]]
    tl_events, lane_events = set(), set()
    for source_id, tl_signal_idx, color in obsereved_events:
        if source_id in traffic_light_ids_all:
            tl_events.add((source_id, (1, 0, 0, 0) if color == TL_GREEN_COLOR else (0, 1, 0, 0)))
        else:
            if color == TL_RED_COLOR:
                lane_events.add((source_id, (0, 0, 1, 0)))
            else:
                lane_events.add((source_id, (0, 0, 0, 1)))
    return list(tl_events), list(lane_events)
        
def run(frame_dataset, dataset_filtered, scene_idx_bounds, output=None, 
        vis=True, show=False, scenes_per_video=10, intersection=None,
        tl_events_df=None
       ):
    tl_signals_buffer = dict()
    timestamp_prev, ego_centroid_prev = datetime(1970, 1, 1).astimezone(timezone('US/Pacific')), np.array([-9999, -9999])
    master_intersection_idx_prev = -9999

    for start_scene in tqdm(range(*scene_idx_bounds, scenes_per_video), desc='Scenes...'):
        end_scene = start_scene + scenes_per_video
        for scene_idx in tqdm(range(start_scene, end_scene), desc=f'scenes {start_scene}-{end_scene}..'):
            # scene_idx = 3
            indexes = frame_dataset.get_scene_indices(scene_idx)
            images = []
            
            seq_order_i = 0
            change_source_id_2_idx = dict()

            for idx in tqdm(indexes, desc='scene indices..'):
                
                centroid = frame_dataset[idx]['ego_centroid']           
                yaw = frame_dataset[idx]['ego_yaw']

                closest_lane_id = find_closest_lane(centroid, yaw, ALL_WHEELS_CLASS, intersections_only=True)
                if closest_lane_id is None or (intersection is not None and lane_id_2_master_intersection_idx[closest_lane_id] != intersection): continue


                info_related_lanes = get_info_per_related_lanes(frame_dataset[idx])
                                
                if len(info_related_lanes):
                
                    master_intersection_idx, timestamp, ego_centroid = info_related_lanes[:3]
                    lane_speeds_completions, tl_faces_info = info_related_lanes[-2:]
                    tl_signals_GO, tl_signals_STOP, observed_events = get_tl_signal_current(lane_speeds_completions, tl_faces_info)
#                     print('='*20+ ' '*5 + str(idx)+ ' '*5 + '='*20)
#                     print(lane_speeds_completions)
#                     print(tl_faces_info)
#                     print(observed_events)
                    tl_signals_buffer = get_accumulated_tl_signals(timestamp, ego_centroid, master_intersection_idx,
                                                                   tl_signals_GO, tl_signals_STOP, 
                                                                   timestamp_prev, ego_centroid_prev, master_intersection_idx_prev,
                                                                   tl_signals_buffer)
#                     print('tl_signals_buffer', tl_signals_buffer)
                    master_intersection_idx_prev = master_intersection_idx
                    timestamp_prev, ego_centroid_prev = timestamp, ego_centroid
                    
                    # plotting
                    # to np source: https://stackoverflow.com/questions/7821518/matplotlib-save-plot-to-numpy-array
                    if vis:
                        data_filtered = dataset_filtered[idx]
                        im_filtered = data_filtered["image"].transpose(1, 2, 0)
                        im_filtered = dataset_filtered.rasterizer.to_rgb(im_filtered)
                        target_positions_pixels_filtered = transform_points(
                            data_filtered["target_positions"] + data_filtered["centroid"][:2],
                            data_filtered["world_to_image"])

                        draw_trajectory(im_filtered, target_positions_pixels_filtered, data_filtered["target_yaws"], TARGET_POINTS_COLOR)
                        im_filtered = im_filtered[::-1, :, :]
                        im_manual = np.zeros_like(im_filtered)

                        fig = plt.figure(figsize=(5, 5))
                        ax = fig.add_subplot(111)
                        plt.axis('off')
                        for lane_id, speed_avg, _, _, completion_min, _ in lane_speeds_completions:
                            plot_line(ax, lane_id, speed_avg, completion_min)
                        for tl_signal_idx, (_, status) in tl_signals_buffer.items():
                            for tl_signal_stop_coord in tl_signal_idx_2_stop_coordinates[tl_signal_idx]:
                                ax.scatter(tl_signal_stop_coord[0], tl_signal_stop_coord[1], c='green' if status == TL_GREEN_COLOR else 'r', s=50)
                                
                        ax.scatter(centroid[0], centroid[1], c='black', s=100)
                        ax.arrow(centroid[0], centroid[1], 3*np.cos(yaw), 3*np.sin(yaw), color='black')
                        ax.set_xlim((centroid[0] - 85, centroid[0] + 85))
                        ax.set_ylim((centroid[1] - 85, centroid[1] + 85))

                        plt.title(f'scene: {scene_idx}, frame: {idx}{" (junction " + str(lane_id_2_master_intersection_idx[closest_lane_id]) + ")" if closest_lane_id in lane_id_2_master_intersection_idx else ""}')
                        fig.canvas.draw()
                        data = np.fromstring(fig.canvas.tostring_rgb(), dtype=np.uint8, sep='').reshape(fig.canvas.get_width_height() + (3,))
                        plt.close()
                        
                        im_plt = cv2.resize(data, (im_filtered.shape[1], im_filtered.shape[0]))
                        im_plt = cv2.resize(im_plt, (im_filtered.shape[1], im_filtered.shape[0]))    
                        im = np.concatenate((imutils.rotate(im_filtered, 180*yaw/np.pi), np.zeros((im_filtered.shape[0], 20, im_filtered.shape[2]), dtype='uint8'), im_plt), axis=1)

                                                               
                        if tl_events_df is not None:
#                             print(master_intersection_idx, timestamp)
                            tl_events_current = tl_events_df.loc[[(master_intersection_idx, timestamp)]]
                            tl_signal_classes = tl_events_current['tl_signal_classes'].values[0]
#                             print('tl_signal_classes from df', tl_signal_classes)
                            rnn_events = tl_events_current['rnn_inputs_raw'].values[0]
                            tl_events = [x for x in rnn_events if x[0] in traffic_light_ids_all]
                            lane_events = [x for x in rnn_events if x[0] in lane_id_2_idx]
                            
#                             print('tl_events, lane_events', tl_events, lane_events)
                            fig = plt.figure(figsize=(5, 5))
                            ax = fig.add_subplot(111)
                            plt.axis('off')
                            for tl_id, ohe_type in tl_events:
                                if tl_id not in change_source_id_2_idx:
                                    change_source_id_2_idx[tl_id] = len(change_source_id_2_idx)
                                color_i = change_source_id_2_idx[tl_id]
                                tl_coord = get_traffic_light_coordinates(tl_id)
                                ax.scatter(tl_coord[0], tl_coord[1], color=cycled_colors[color_i % len(cycled_colors)], s=50)
#                                 ax.text(tl_signal_stop_coord[0] + 1.7, tl_signal_stop_coord[1] + 5, 
#                                         f'  seq order: {seq_order_i}', 
#                                         fontsize=14)
                                ohe_based_color_check = None
                                if ohe_type[0] == 1 and np.sum(ohe_type) == 1:
                                    ohe_based_color_check = 'green'
                                elif ohe_type[1] == 1 and np.sum(ohe_type) == 1:
                                    ohe_based_color_check = 'red'
                                else:
                                    ohe_based_color_check = 'BUG!!!!'
                                ax.text(tl_signal_stop_coord[0] + 1.7, tl_signal_stop_coord[1] - 5, 
                                        ohe_based_color_check, 
                                        color=cycled_colors[color_i % len(cycled_colors)],
                                        fontsize=12)
                                seq_order_i += 1  
                                
                            for lane_id, ohe_type in lane_events:
                                if lane_id not in change_source_id_2_idx:
                                    change_source_id_2_idx[lane_id] = len(change_source_id_2_idx)
                                color_i = change_source_id_2_idx[lane_id]
                                seq_order_text = ''#f'  seq order: {seq_order_i}'
                                ohe_based_text_check = None
                                if ohe_type[2] == 1 and np.sum(ohe_type) == 1:
                                    ohe_based_text_check = 'idle'
                                elif ohe_type[3] == 1 and np.sum(ohe_type) == 1:
                                    ohe_based_text_check = 'on move'
                                else:
                                    ohe_based_text_check = 'BUG!!!!'
                                for j, lane_id_ in enumerate(lane_id.split('_')):
                                    plot_line(ax, lane_id_, color=cycled_colors[color_i % len(cycled_colors)], text1=seq_order_text if j==0 else '', 
                                              text2=ohe_based_text_check)
                                seq_order_i += 1

                            ax.set_xlim((centroid[0] - 85, centroid[0] + 85))
                            ax.set_ylim((centroid[1] - 85, centroid[1] + 85))

                            plt.title(f'Inputs')
                            fig.canvas.draw()
                            data_X = np.fromstring(fig.canvas.tostring_rgb(), dtype=np.uint8, sep='').reshape(fig.canvas.get_width_height() + (3,))
                            plt.close()

                            time_to_tl_change = tl_events_current['time_to_tl_change'].values[0]
                            fig = plt.figure(figsize=(5, 5))
                            ax = fig.add_subplot(111)
                            plt.axis('off')
                            tl_sig_idx_2_min_x, tl_sig_idx_2_min_y = dict(), dict()
                            for tl_signal_order_num, (tl_signal_idx, color) in enumerate(tl_signal_classes.items()):
                                color_vis = cycled_colors_targets[tl_signal_idx % len(cycled_colors_targets)]
#                                     print(tl_signal_idx, color_vis)
                                for tl_signal_stop_coord in tl_signal_idx_2_stop_coordinates[tl_signal_idx]:
                                    ax.scatter(tl_signal_stop_coord[0], tl_signal_stop_coord[1], color='green' if color==TL_GREEN_COLOR else 'r', s=70)
                                    ax.scatter(tl_signal_stop_coord[0], tl_signal_stop_coord[1], color=color_vis, s=30)
                                    if tl_signal_idx not in tl_sig_idx_2_min_x:
                                        tl_sig_idx_2_min_x[tl_signal_idx] = tl_signal_stop_coord[0]
                                        tl_sig_idx_2_min_y[tl_signal_idx] = tl_signal_stop_coord[1]
                                    else:
                                        tl_sig_idx_2_min_x[tl_signal_idx] = min(tl_signal_stop_coord[0], tl_sig_idx_2_min_x[tl_signal_idx])
                                        tl_sig_idx_2_min_y[tl_signal_idx] = min(tl_signal_stop_coord[1], tl_sig_idx_2_min_y[tl_signal_idx])


                                if tl_signal_idx in time_to_tl_change:
                                    tte = time_to_tl_change[tl_signal_idx]
                                    text_coord = (centroid[0] - 85, centroid[1] - 85 + 14.5*tl_signal_order_num)
                                    ax.text(*text_coord,  
                                            f'TTE: {tte:.2f} sec', color=color_vis, 
                                    fontsize=14)
                                    ax.arrow(*text_coord, 
                                             tl_sig_idx_2_min_x[tl_signal_idx] - text_coord[0],  
                                             tl_sig_idx_2_min_y[tl_signal_idx] - text_coord[1],
                                             color=color_vis)


                            ax.set_xlim((centroid[0] - 85, centroid[0] + 85))
                            ax.set_ylim((centroid[1] - 85, centroid[1] + 85))

                            plt.title(f'Targets')
                            fig.canvas.draw()
                            data_y = np.fromstring(fig.canvas.tostring_rgb(), dtype=np.uint8, sep='').reshape(fig.canvas.get_width_height() + (3,))
                            plt.close()

                            im_X = cv2.resize(data_X, (im_filtered.shape[1], im_filtered.shape[0]))   

                            im_y = cv2.resize(data_y, (im_filtered.shape[1], im_filtered.shape[0]))

                            im_xy = np.concatenate((im_X, np.zeros((im_filtered.shape[0], 20, im_filtered.shape[2]), dtype='uint8'), im_y), axis=1)

                            im_ = np.concatenate((im, np.zeros((20, im.shape[1], im.shape[2]), dtype='uint8'), im_xy), axis=0)
                            output = store_output(im_, output, start_scene, end_scene, intersection, show)
                        
                        if tl_events_df is None:
                            output = store_output(im, output, start_scene, end_scene, intersection, show)
                        

        if output is not None:
            output.release()
            output = None
        print('closed...')

In [102]:
for intersection in range(10):
    run(frame_dataset, dataset_filtered, [0, 200], show=False, scenes_per_video=20, intersection=intersection, tl_events_df=tl_events_df.set_index(['master_intersection_idx', 'timestamp']))
# run(frame_dataset, dataset_filtered, [0, 200], show=False, scenes_per_video=5)#, intersection=intersection)
# run(frame_dataset, dataset_filtered, [29, 30], show=True, scenes_per_video=1, intersection=4)
# run(frame_dataset, dataset_filtered, [27, 28], show=True, scenes_per_video=1, intersection=4) #, tl_events_df=tl_events_df.set_index(['master_intersection_idx', 'timestamp']))



closed...




closed...




closed...




closed...




closed...


...



closed...




closed...




closed...




closed...


KeyboardInterrupt: 

In [80]:
# master_intersection_check = 4
# timestamp_check = datetime(2019, 12, 13, 17, 20, 40, 2828).astimezone(timezone('US/Pacific'))
# pd.set_option('display.max_rows', 5000)
# tl_events_df_check = tl_events_df[(tl_events_df['master_intersection_idx'] == master_intersection_check) &
#                                   (tl_events_df['timestamp'] >= timestamp_check - timedelta(minutes=70)) &
#                                   (tl_events_df['timestamp'] <= timestamp_check + timedelta(minutes=70))]
# tl_events_df_check.head()

In [84]:
def tl_seq_collate_fn(frames_batch):
    batch_result = []
    for frame in frames_batch:
        info_related_lanes = get_info_per_related_lanes(frame)
        if len(info_related_lanes):
            master_intersection_idx, timestamp, ego_centroid, ego_yaw, lanes_info, tl_faces_info = info_related_lanes
            tl_signals_GO, tl_signals_STOP, observed_events = get_tl_signal_current(lanes_info, tl_faces_info)
            batch_result.append((master_intersection_idx, timestamp, ego_centroid, observed_events, tl_signals_GO, tl_signals_STOP))
    return np.array(batch_result)

dataloader_frames = DataLoader(frame_dataset, shuffle=False, batch_size=100,
                               num_workers=28,
                               collate_fn=tl_seq_collate_fn) 

In [85]:
master_intersection_idx_2_tl_signal_indices

defaultdict(list,
            {0: [0, 8, 9, 56, 57, 58, 59],
             3: [1, 14, 22, 28, 37, 38, 39],
             8: [2, 11, 32, 34, 60],
             1: [3, 12, 20, 24, 30, 33, 54],
             6: [4, 6, 41, 42, 43],
             4: [5, 13, 23, 31, 49, 50, 51, 61, 62],
             7: [7, 15, 44, 45, 46, 48, 52, 53],
             2: [10, 21, 36, 47, 55],
             9: [16, 18, 27, 40],
             5: [17, 19, 25, 26, 29, 35]})

In [86]:
master_intersection_idx_list, timestamp_list, ego_centroid_list, observed_events_list, tl_signals_GO_list, tl_signals_STOP_list = [[] for _ in range(6)]
for batch in tqdm(dataloader_frames):
    for record in batch:
        master_intersection_idx, timestamp, ego_centroid, observed_events, tl_signals_GO, tl_signals_STOP = record
        master_intersection_idx_list.append(master_intersection_idx)
        timestamp_list.append(timestamp)
        ego_centroid_list.append(ego_centroid)
        observed_events_list.append(observed_events)
        tl_signals_GO_list.append(tl_signals_GO)
        tl_signals_STOP_list.append(tl_signals_STOP)

In [87]:
tl_events_df = pd.DataFrame({'master_intersection_idx': master_intersection_idx_list, 
                             'timestamp': timestamp_list, 
                             'ego_centroid': ego_centroid_list, 
                             'observed_events': observed_events_list, 
                             'tl_signals_GO': tl_signals_GO_list, 
                             'tl_signals_STOP': tl_signals_STOP_list})

In [88]:
tl_events_df.sort_values(by=['master_intersection_idx', 'timestamp'], inplace=True)

In [89]:
np.sum([len(x) == 0 for x in tl_signals_GO_list])

160990

In [90]:
tl_events_df['tl_signals_GO'].map(len).value_counts()

1    177415
0    160990
2     82473
3      6056
4       588
Name: tl_signals_GO, dtype: int64

In [91]:
def compute_tl_signal_classes(tl_events_df):
    tl_signals_buffer = dict()
    timestamp_prev, ego_centroid_prev = datetime(1970, 1, 1).astimezone(timezone('US/Pacific')), np.array([-9999, -9999])
    master_intersection_idx_prev = -9999
    tl_events_df['tl_signal_classes'] = [dict() for _ in range(len(tl_events_df))]
    for row_index, row in tqdm(tl_events_df.iterrows(), total=len(tl_events_df), desc="Cumul tl's.."):

        tl_signals_buffer = get_accumulated_tl_signals(row['timestamp'], 
                                                         row['ego_centroid'],
                                                         row['master_intersection_idx'],
                                                         row['tl_signals_GO'], 
                                                         row['tl_signals_STOP'], 
                                                         timestamp_prev, 
                                                       ego_centroid_prev, 
                                                       master_intersection_idx_prev,
                                                       tl_signals_buffer)
        timestamp_prev = row['timestamp']                                                     
        ego_centroid_prev = row['ego_centroid']
        master_intersection_idx_prev = row['master_intersection_idx']
        tl_events_df.loc[row_index]['tl_signal_classes'].update({key:val[1] for key, val in tl_signals_buffer.items()})
compute_tl_signal_classes(tl_events_df)

In [92]:
def compute_time_to_tl_change(tl_events_df, ego_translation_m_max=1.5, time_to_event_ub=5.01, verbose=False):
    active_next_tl_signals = dict() # tl_sig_idx -> color_code
    relevant_tl_signal_change_event = dict() # tl_sig_idx -> (event_timestamp, new_event_color_code)
    timestamp_next, ego_centroid_next = datetime(2050, 1, 1).astimezone(timezone('US/Pacific')), np.array([-9999, -9999])
    master_intersection_idx_next = -9999
    tl_events_df['time_to_tl_change'] = [dict() for _ in range(len(tl_events_df))]
    timediff_max_sec = 1
    the_same_color_duration = dict()

    for row_i in tqdm(range(len(tl_events_df) - 1, -1, -1), desc='Time to color change..'):
        row = tl_events_df.iloc[row_i]

        # check that we have the same ego sdv, the same intersection, and consec. frame
        if row['master_intersection_idx'] != master_intersection_idx_next:
            active_next_tl_signals = dict()
            relevant_tl_signal_change_event = dict()
            for tl_signal_idx, color_code in row['tl_signal_classes'].items():
                relevant_tl_signal_change_event[tl_signal_idx] = (row['timestamp'], color_code)
                active_next_tl_signals[tl_signal_idx] = color_code
            the_same_color_duration = dict()
            if verbose:
                print('intersection reset')
        elif (timestamp_next - row['timestamp']).total_seconds() > timediff_max_sec:
            active_next_tl_signals = dict()
            relevant_tl_signal_change_event = dict()
            for tl_signal_idx, color_code in row['tl_signal_classes'].items():
                relevant_tl_signal_change_event[tl_signal_idx] = (row['timestamp'], color_code)
                active_next_tl_signals[tl_signal_idx] = color_code
            the_same_color_duration = dict()
            if verbose:
                print('time reset')
        else:
            translation_m = np.hypot(ego_centroid_next[0] - row['ego_centroid'][0], ego_centroid_next[1] - row['ego_centroid'][1])
            if translation_m > ego_translation_m_max:
                active_next_tl_signals = dict()
                relevant_tl_signal_change_event = dict()
                # going backward in time, row records would be future for the prev rows
                for tl_signal_idx, color_code in row['tl_signal_classes'].items():
                    relevant_tl_signal_change_event[tl_signal_idx] = (row['timestamp'], color_code)
                    active_next_tl_signals[tl_signal_idx] = color_code
                the_same_color_duration = dict()
                if verbose:
                    print('translation reset')

        if verbose:
            print('tl_signal_classes', row['tl_signal_classes'])
        # if tl signal class (color) is currently unknown, then there's no reliable way to set the corresponding change event
        relevant_tl_signal_change_event_items = list(relevant_tl_signal_change_event.items())
        for tl_signal_idx, _ in relevant_tl_signal_change_event_items:
            if tl_signal_idx not in row['tl_signal_classes']:
                del relevant_tl_signal_change_event[tl_signal_idx]
                if verbose:
                    print(f'removing {tl_signal_idx} as currently non-present')
                    
#         # updating change events by comparison of current timestamp and the next one
#         relevant_tl_signal_change_event_items = list(relevant_tl_signal_change_event.items())
#         for tl_signal_idx, (event_timestamp, new_event_color_code) in relevant_tl_signal_change_event_items:
#             # if the event stored in relevant_tl_signal_change_event is not the latest change event, cause the latest change happens in the following timestamp
#             if active_next_tl_signals[tl_signal_idx] != row['tl_signal_classes'][tl_signal_idx]:
#                 relevant_tl_signal_change_event[tl_signal_idx] = (active_next_tl_signals['timestamp'], active_next_tl_signals['tl_signal_classes'][tl_signal_idx])
                    
        time_to_tl_change_current = dict()
        
        if verbose:
            print('relevant_tl_signal_change_event', relevant_tl_signal_change_event)
            
        relevant_tl_signal_change_event_items = list(relevant_tl_signal_change_event.items())
        for tl_signal_idx, (event_timestamp, event_color_code) in relevant_tl_signal_change_event_items:
            # if the current tl color is the same, as in the future change event, ...
            if event_color_code != row['tl_signal_classes'][tl_signal_idx]:
                time_to_tl_change_current[tl_signal_idx] = np.clip((event_timestamp - row['timestamp']).total_seconds(), 0, time_to_event_ub)
                if verbose:
                    print(f'tl_signal_idx: {tl_signal_idx} -> diff color as event {(event_timestamp, event_color_code)}, tte: {time_to_tl_change_current[tl_signal_idx]}')
            else:
                # check color continuity
                if active_next_tl_signals[tl_signal_idx] == row['tl_signal_classes'][tl_signal_idx]:
                    if verbose:
                        print(f'tl_signal_idx: {tl_signal_idx} the same color, dicrement event start')
            
                    relevant_tl_signal_change_event[tl_signal_idx] = (row['timestamp'], event_color_code)
                    if tl_signal_idx not in the_same_color_duration:
                        the_same_color_duration[tl_signal_idx] = row['timestamp']
                    elif (the_same_color_duration[tl_signal_idx] - row['timestamp']).total_seconds() >= time_to_event_ub:
                        time_to_tl_change_current[tl_signal_idx] = time_to_event_ub
                        if verbose:
                            print(f'too long the same color: TTE at least 5.01')
                else:
                    # the event stored in relevant_tl_signal_change_event is not the latest change event, cause the latest change happens in the following timestamp
                    relevant_tl_signal_change_event[tl_signal_idx] = (timestamp_next, active_next_tl_signals[tl_signal_idx])
                    time_to_tl_change_current[tl_signal_idx] = np.clip((timestamp_next - row['timestamp']).total_seconds(), 0, time_to_event_ub)
                    
        active_next_tl_signals = row['tl_signal_classes'].copy() 
        for tl_signal_idx, color_code in row['tl_signal_classes'].items():
            if tl_signal_idx not in relevant_tl_signal_change_event:
                relevant_tl_signal_change_event[tl_signal_idx] = (row['timestamp'], color_code)
        
        timestamp_next, ego_centroid_next, master_intersection_idx_next = row['timestamp'], row['ego_centroid'], row['master_intersection_idx']
        tl_events_df.iloc[row_i]['time_to_tl_change'].update(time_to_tl_change_current)
# compute_time_to_tl_change(tl_events_df_check, verbose=True)
# tl_events_df_check
compute_time_to_tl_change(tl_events_df)

In [112]:
for p in np.arange(0, 1, 0.05):
    print(p, -np.log(p))

0.0 inf
0.05 2.995732273553991
0.1 2.3025850929940455
0.15000000000000002 1.897119984885881
0.2 1.6094379124341003
0.25 1.3862943611198906
0.30000000000000004 1.203972804325936
0.35000000000000003 1.0498221244986776
0.4 0.916290731874155
0.45 0.7985076962177716
0.5 0.6931471805599453
0.55 0.5978370007556204
0.6000000000000001 0.5108256237659905
0.65 0.4307829160924542
0.7000000000000001 0.3566749439387323
0.75 0.2876820724517809
0.8 0.2231435513142097
0.8500000000000001 0.1625189294977748
0.9 0.10536051565782628
0.9500000000000001 0.05129329438755046


In [93]:
def compute_rnn_inputs(tl_events_df):
    tl_events_df['rnn_inputs_raw'] = [[] for _ in range(len(tl_events_df))]
    for row_idx, row in tl_events_df.iterrows():
        tl_inputs, lane_inputs = get_rnn_inputs_from_events(row['observed_events'])
        tl_events_df.loc[row_idx]['rnn_inputs_raw'].extend(tl_inputs + lane_inputs)
compute_rnn_inputs(tl_events_df)

In [94]:
len(tl_events_df)

427522

In [108]:
tl_events_df

,master_intersection_idx,timestamp,ego_centroid,observed_events,tl_signals_GO,tl_signals_STOP,tl_signal_classes,time_to_tl_change,rnn_inputs_raw
542589,0,2019-11-21 11:28:36.903368-08:00,"[753.0741577148438, -1408.6817626953125]","{(NTTe, 8, 0), (NTTe, 9, 0)}",{},"{8, 9}","{8: 0, 9: 0}","{8: 5.01, 9: 5.01}","[(NTTe, (0, 1, 0, 0))]"
542590,0,2019-11-21 11:28:37.003465-08:00,"[752.4585571289062, -1408.2960205078125]","{(NTTe, 8, 0), (NTTe, 9, 0)}",{},"{8, 9}","{8: 0, 9: 0}","{8: 5.01, 9: 5.01}","[(NTTe, (0, 1, 0, 0))]"
542591,0,2019-11-21 11:28:37.103479-08:00,"[751.8576049804688, -1407.9105224609375]","{(NTTe, 8, 0), (NTTe, 9, 0)}",{},"{8, 9}","{8: 0, 9: 0}","{8: 5.01, 9: 5.01}","[(NTTe, (0, 1, 0, 0))]"
542592,0,2019-11-21 11:28:37.203406-08:00,"[751.2593994140625, -1407.535888671875]","{(NTTe, 8, 0), (NTTe, 9, 0), (/ggb, 9, 0), (/g...",{},"{8, 9}","{8: 0, 9: 0}","{8: 5.01, 9: 5.01}","[(/ggb, (0, 1, 0, 0)), (NTTe, (0, 1, 0, 0))]"
542593,0,2019-11-21 11:28:37.303268-08:00,"[750.6685180664062, -1407.1754150390625]","{(NTTe, 8, 0), (NTTe, 9, 0)}",{},"{8, 9}","{8: 0, 9: 0}","{8: 5.01, 9: 5.01}","[(NTTe, (0, 1, 0, 0))]"
...,...,...,...,...,...,...,...,...,...
1308765,9,2020-03-13 15:36:54.103391-07:00,"[-30.599111557006836, 339.60968017578125]","{(LqrC, 16, 1), (LqrC_block, 40, 0), (xT+a, 18...","{16, 18}","{40, 27}","{18: 1, 16: 1, 40: 0, 27: 0}",{},"[(xT+a, (0, 0, 0, 1)), (LqrC, (0, 0, 0, 1))]"
1308766,9,2020-03-13 15:36:54.203124-07:00,"[-29.680238723754883, 338.59796142578125]","{(LqrC, 16, 1), (LqrC_block, 40, 0), (xT+a, 18...","{16, 18}","{40, 27}","{18: 1, 16: 1, 40: 0, 27: 0}",{},"[(xT+a, (0, 0, 0, 1)), (LqrC, (0, 0, 0, 1))]"
1308767,9,2020-03-13 15:36:54.302850-07:00,"[-28.809247970581055, 337.6002197265625]","{(LqrC, 16, 1), (LqrC_block, 40, 0), (LqrC_blo...",{16},"{40, 27}","{18: 1, 16: 1, 40: 0, 27: 0}",{},"[(LqrC, (0, 0, 0, 1))]"
1308768,9,2020-03-13 15:36:54.402543-07:00,"[-27.933822631835938, 336.6076965332031]","{(LqrC, 16, 1), (LqrC_block, 40, 0), (LqrC_blo...",{16},"{40, 27}","{18: 1, 16: 1, 40: 0, 27: 0}",{},"[(LqrC, (0, 0, 0, 1))]"


In [109]:
tl_events_df[['master_intersection_idx', 'rnn_inputs_raw', 'tl_signal_classes', 'time_to_tl_change']].to_hdf('../input/tl_events_df_val.hdf5', key='data')

In [113]:
for intersection in range(4, 10):
    run(frame_dataset, dataset_filtered, [0, 80], show=False, scenes_per_video=10, intersection=intersection, tl_events_df=tl_events_df.set_index(['master_intersection_idx', 'timestamp']))



closed...




closed...




closed...




closed...




closed...


...



closed...




closed...




closed...




closed...




closed...



In [104]:
tl_events_df = pd.read_hdf('../input/tl_events_df_train_full_0.hdf5', key='data')

In [107]:
tl_events_df_val = pd.read_hdf('../input/tl_events_df_val.hdf5', key='data')

In [111]:
tl_events_df_val[['master_intersection_idx', 'timestamp', 'rnn_inputs_raw', 'tl_signal_classes', 'time_to_tl_change']].to_hdf('../input/tl_events_df_val.hdf5', key='data')

In [105]:
len(tl_events_df)

1310954

In [108]:
len(tl_events_df_val)

1494704

In [109]:
tl_events_df_val.head()

,master_intersection_idx,timestamp,rnn_inputs_raw,tl_signal_classes,time_to_tl_change
542589,0,2019-11-21 11:28:36.903368-08:00,"[(NTTe, (0, 1, 0, 0))]","{8: 0, 9: 0}","{8: 5.01, 9: 5.01}"
542590,0,2019-11-21 11:28:37.003465-08:00,"[(NTTe, (0, 1, 0, 0))]","{8: 0, 9: 0}","{8: 5.01, 9: 5.01}"
542591,0,2019-11-21 11:28:37.103479-08:00,"[(NTTe, (0, 1, 0, 0))]","{8: 0, 9: 0}","{8: 5.01, 9: 5.01}"
542592,0,2019-11-21 11:28:37.203406-08:00,"[(NTTe, (0, 1, 0, 0)), (/ggb, (0, 1, 0, 0))]","{8: 0, 9: 0}","{8: 5.01, 9: 5.01}"
542593,0,2019-11-21 11:28:37.303268-08:00,"[(NTTe, (0, 1, 0, 0))]","{8: 0, 9: 0}","{8: 5.01, 9: 5.01}"


In [106]:
tl_events_df.head()

,master_intersection_idx,timestamp,rnn_inputs_raw,tl_signal_classes,time_to_tl_change
4781,0,2019-10-14 13:57:25.502484-07:00,"[(/ggb, (0, 1, 0, 0)), (NTTe, (0, 1, 0, 0))]","{8: 0, 9: 0}","{8: 5.01, 9: 5.01}"
4782,0,2019-10-14 13:57:25.602662-07:00,"[(/ggb, (0, 1, 0, 0)), (NTTe, (0, 1, 0, 0))]","{8: 0, 9: 0}","{8: 5.01, 9: 5.01}"
4783,0,2019-10-14 13:57:25.702806-07:00,"[(NTTe, (0, 1, 0, 0))]","{8: 0, 9: 0}","{8: 5.01, 9: 5.01}"
4784,0,2019-10-14 13:57:25.802895-07:00,"[(/ggb, (0, 1, 0, 0)), (NTTe, (0, 1, 0, 0))]","{8: 0, 9: 0}","{8: 5.01, 9: 5.01}"
4785,0,2019-10-14 13:57:25.902922-07:00,"[(/ggb, (0, 1, 0, 0)), (NTTe, (0, 1, 0, 0))]","{8: 0, 9: 0}","{8: 5.01, 9: 5.01}"


In [80]:
import cProfile
import pstats

cProfile.runctx('run(frame_dataset, dataset_filtered, scene_idx_bounds)', 
                {'run': run, 'frame_dataset': frame_dataset, 'dataset_filtered': dataset_filtered, 'scene_idx_bounds': [4,6]},
                {}, 'tl_data_gen_profilling')

KeyboardInterrupt: 

Error in callback <function flush_figures at 0x7f8e65434cb0> (for post_execute):


KeyboardInterrupt: 

In [ ]:
p = pstats.Stats('tl_data_gen_profilling')
p.strip_dirs().sort_stats('cumulative').print_stats(30)

In [ ]:
plt.figure(figsize=(19, 19))

for master_intersection_i, master_intersection_tlights in enumerate(master_intersection_idx_2_traffic_lights):
    master_x_list, master_y_list = [], []
    for tl_id in master_intersection_tlights:
        tl_coord = traffic_light_id_2_coord[tl_id]
        master_x_list.append(tl_coord[0])
        master_y_list.append(tl_coord[1])
        plt.scatter(tl_coord[0], tl_coord[1], c='b', s=10)
    plt.scatter(np.mean(master_x_list), np.mean(master_y_list), s=70, label=f'{master_intersection_i}')
plt.legend()


lane_id = 'Ib85'
lane_center_line = lanes_crosswalks['lanes']['center_line'][lane_id_2_idx[lane_id]]
lane_left_board = lanes_crosswalks['lanes']['xy_left_'][lane_id_2_idx[lane_id]]
lane_right_board = lanes_crosswalks['lanes']['xy_right_'][lane_id_2_idx[lane_id]]    
plt.scatter(lane_center_line[:, 0], lane_center_line[:, 1], s=110)    
plt.scatter(lane_left_board[:, 0], lane_left_board[:, 1], s=1, color='grey')    
plt.scatter(lane_right_board[:, 0], lane_right_board[:, 1], s=1, color='grey')

for frame_idx in indices_of_interest:
    frame_ego_centroid = frame_dataset[frame_idx]['ego_centroid']
    plt.scatter(frame_ego_centroid[0], frame_ego_centroid[1], s=300, c='r')
                
plt.legend()